# CMS GIWAXS plotting notebook - plotting single images from loaded zarr datasets
# PM6 Y6-series solvent study GIWAXS CMS 2024C2

## Imports

In [ ]:
# Imports:
import pathlib
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import xarray as xr
from tqdm.auto import tqdm

# Choose a colormap:
cmap = plt.cm.turbo
cmap.set_bad('black')

In [ ]:
from scipy import optimize, signal
from lmfit import models

## Define & check paths

In [ ]:
def select_attrs(data_arrays_iterable, selected_attrs_dict):
    """
    Selects data arrays whose attributes match the specified values.

    Parameters:
    data_arrays_iterable: Iterable of xarray.DataArray objects.
    selected_attrs_dict: Dictionary where keys are attribute names and 
                         values are the attributes' desired values.

    Returns:
    List of xarray.DataArray objects that match the specified attributes.
    """    
    sublist = list(data_arrays_iterable)
    
    for attr_name, attr_values in selected_attrs_dict.items():
        sublist = [da.copy() for da in sublist if da.attrs[attr_name] in attr_values]
                
    return sublist

def fold_image(data_array, fold_axis, progress_bar=False):
    """
    Method to fold image along a specified axis.
    
    Parameters:
    - data_array (xarray DataArray): The DataArray to fold
    - fold_axis (str): The axis along which to fold the image
    
    Returns:
    - xarray DataArray: The folded image
    """
    # Filter data for fold_axis >= 0 and fold_axis <= 0
    positive_data = data_array.where(data_array[fold_axis] >= 0, drop=True)
    negative_data = data_array.where(data_array[fold_axis] <= 0, drop=True)
    
    # Reverse negative_data for easier comparison
    negative_data = negative_data.reindex({fold_axis: negative_data[fold_axis][::-1]})
    
    # Find the maximum coordinate of the shorter quadrant (positive_data)
    max_positive_coord = float(positive_data[fold_axis].max())
    
    # Find the equivalent coordinate in the negative_data
    abs_diff = np.abs(negative_data[fold_axis].values + max_positive_coord)
    
    # Minimize the difference
    min_diff_idx = np.argmin(abs_diff)
    
    # Check if the lengths are equivalent
    len_pos = len(positive_data[fold_axis])
    len_neg = len(negative_data[fold_axis][:min_diff_idx+1])
    
    if len_pos != len_neg:
        # Adjust the coordinate range for negative_data
        for i in range(1, 4):  # Check 3 neighbors
            new_idx = min_diff_idx + i
            len_neg = len(negative_data[fold_axis][:new_idx+1])
            if len_pos == len_neg:
                min_diff_idx = new_idx
                break
                
    # Crop the negative_data to match positive_data length
    negative_data_cropped = negative_data.isel({fold_axis: slice(0, min_diff_idx+1)})
    
    # Prepare the new data array
    new_data = xr.zeros_like(positive_data)
    
    # Fold the image
    if progress_bar:
        for i in tqdm(range(len(positive_data[fold_axis]))):
            pos_val = positive_data.isel({fold_axis: i}).values
            neg_val = negative_data_cropped.isel({fold_axis: i}).values

            # Pixel comparison and choosing 
            new_data[i] = np.where(((pos_val == 0) | (np.isnan(pos_val))) & (neg_val > 1-1e-6),
                                   neg_val, 
                                   pos_val)
    else:
        for i in range(len(positive_data[fold_axis])):
            pos_val = positive_data.isel({fold_axis: i}).values
            neg_val = negative_data_cropped.isel({fold_axis: i}).values

            # Pixel comparison and choosing 
            new_data[i] = np.where(((pos_val == 0) | (np.isnan(pos_val)) | (pos_val < 2)) & (neg_val > 0.001),
                                   neg_val, 
                                   pos_val) 
            
            # # Pixel comparison and averaging
            # new_data[i] = np.where(
            #     (pos_val > 0) & (neg_val > 0), 
            #     (pos_val + neg_val) / 2,
            #     np.where(((pos_val == 0) | (np.isnan(pos_val))) & (neg_val > 0),
            #              neg_val, 
            #              pos_val)
            # )


        
    # Append residual data from the longer quadrant if exists
    if len(negative_data[fold_axis]) > min_diff_idx+1:
        residual_data = negative_data.isel({fold_axis: slice(min_diff_idx+1, None)})
        residual_data[fold_axis] = np.abs(residual_data[fold_axis])
        new_data = xr.concat([new_data, residual_data], dim=fold_axis)
        
    # Update data_array with the folded image
    data_array = new_data.sortby(fold_axis)
    
    # Inherit coordinates and metadata attributes from the original data_array
    data_array.attrs = data_array.attrs.copy()
    data_array.attrs['fold_axis'] = fold_axis  # Add 'fold_axis' attribute

    # Ensure all original coordinates are retained in the new data_array
    for coord in data_array.coords:
        if coord not in data_array.coords:
            data_array = data_array.assign_coords({coord: data_array[coord]})

    return data_array

In [ ]:
# I like pathlib for its readability & checkability, it's also necessary for the loadSeries function later on
# Replace the paths with the ones relevant to your data, you can use the ".exists()" method to make sure you defined a path correctly
userPath = pathlib.Path('/nsls2/users/alevin')
propPath = pathlib.Path('/nsls2/data/cms/proposals/2023-3/pass-311415')
zarrsPath = propPath.joinpath('AL_2024C2/processed_data/zarrs')
outPath = propPath.joinpath('AL_2024C2/processed_data')

In [ ]:
# List zarrs
sorted([f.name for f in zarrsPath.iterdir()])  # a way to list just the filenames and not the whole path

In [ ]:
# Corrected with flipped bar
sn = {
    "AL01": "Y6",
    "AL02": "Y6:PVK 1:1",
    "AL03": "Y6:PVK 1:9",
    "AL04": "A1",
    "AL05": "A1:PVK 1:1",
    "AL06": "A1:PVK 1:9",
    "AL07": "A2",
    "AL08": "A2:PVK 1:1",
    "AL09": "A2:PVK 1:9",
    "AL10": "A3",
    "AL11": "A3:PVK 1:1",
    "AL12": "A3:PVK 1:9",
    "AL13": "Y6 CF:CB 4:1",
    "AL14": "Y6 CF:CB 2:3",
    "AL15": "Y6 CF:CB 2:3 + 0.5% CN",
    "AL16": "PM6 CF:CB 4:1",
    "AL17": "PM6 CF:CB 2:3",
    "AL18": "PM6 CF:CB 2:3 + 0.5% CN",
    "AL19": "PM6:Y6 CF:CB 4:1",
    "AL20": "PM6:Y6 CF:CB 2:3",
    "AL21": "PM6:Y6 CF:CB 2:3 + 0.5% CN",
    "AL22": "PM6 CB",
    "AL23": "Y6 CB",
    "AL24": "Y6BO CB",
    "AL25": "PM6 CB + 1% CN",
    "AL26": "PM6 CB + 5% CN",
    "AL27": "Y6 CB + 0.5% CN",
    
    "AL28": "PM6 CF + 1% CN",
    "AL29": "Y6BO CF",
    "AL30": "Y6 CF",
    "AL31": "PM6 CF",
    "AL32": "PM6:Y6BO CB + 0.5% CN",
    "AL33": "PM6:Y6 CB + 0.5% CN",
    "AL34": "PM6:Y6BO CB",
    "AL35": "PM6:Y6 CB",
    "AL36": "PM6 CB + 0.5% CN",
    "AL37": "Y6BO CB + 0.5% CN",
    
    "AL38": "PM6 CF + 5% CN",
    "AL39": "Y6 CF + 0.5% CN",
    "AL40": "Y6BO CF + 0.5% CN",
    "AL41": "PM6 CF + 0.5% CN",
    "AL42": "PM6:Y6 CF",
    "AL43": "PM6:Y6BO CF",
    "AL44": "PM6:Y6 CF + 0.5% CN",
    "AL45": "PM6:Y6BO CF + 0.5% CN"
}

sn_id = {
    "AL01": "Y6-PVK_1-0",
    "AL02": "Y6-PVK_1-1",
    "AL03": "Y6-PVK_1-9",
    "AL04": "A1-PVK_1-0",
    "AL05": "A1-PVK_1-1",
    "AL06": "A1-PVK_1-9",
    "AL07": "A2-PVK_1-0",
    "AL08": "A2-PVK_1-1",
    "AL09": "A2-PVK_1-9",
    "AL10": "A3-PVK_1-0",
    "AL11": "A3-PVK_1-1",
    "AL12": "A3-PVK_1-9",
    "AL13": "Y6_4CF-1CB",
    "AL14": "Y6_2CF-3CB",
    "AL15": "Y6_p5CN-2CF-3CB",
    "AL16": "PM6_4CF-1CB",
    "AL17": "PM6_2CF-3CB",
    "AL18": "PM6_p5CN-2CF-3CB",
    "AL19": "PM6-Y6_4CF-1CB",
    "AL20": "PM6-Y6_2CF-3CB",
    "AL21": "PM6-Y6_p5CN-2CF-3CB",
    "AL22": "PM6_0CN-CB",
    "AL23": "Y6_CB",
    "AL24": "Y6BO_CB",
    "AL25": "PM6_1CN-CB",
    "AL26": "PM6_5CN-CB",
    "AL27": "Y6_p5CN-CB",
    
    
    "AL28": "PM6_1CN-CF",
    "AL29": "Y6BO_CF",
    "AL30": "Y6_CF",
    "AL31": "PM6_0CN-CF",
    "AL32": "PM6-Y6BO_p5CN-CB",
    "AL33": "PM6-Y6_p5CN-CB",
    "AL34": "PM6-Y6BO_CB",
    "AL35": "PM6-Y6_CB",
    "AL36": "PM6_p5CN-CB",
    "AL37": "Y6BO_p5CN-CB",
    
    "AL38": "PM6_5CN-CF",    
    "AL39": "Y6_p5CN-CF",
    "AL40": "Y6BO_p5CN-CF",
    "AL41": "PM6_p5CN-CF",
    "AL42": "PM6-Y6_CF",
    "AL43": "PM6-Y6BO_CF",
    "AL44": "PM6-Y6_p5CN-CF",
    "AL45": "PM6-Y6BO_p5CN-CF"
}

## Polar plots

### Load dataset

In [ ]:
[f.name for f in zarrsPath.glob('*')]

In [ ]:
filename = 'caked_DS.zarr'
DS = xr.open_zarr(zarrsPath.joinpath(filename))
DS = DS.where(DS>1e-6)
DS

In [ ]:
### Apply a sin chi correction
sin_chi_DA = np.sin(np.radians(np.abs(DS.chi)))

corr_DS = DS.copy()
# corr_DS = corr_DS * sin_chi_DA  # This works mathematically, but does not preserve attributes
for var in corr_DS.data_vars:
    corrected = corr_DS[var] * sin_chi_DA
    corr_DS[var].values = corrected.values
    
corr_DS

In [ ]:
### Fold sin(chi) corrected dataset

folded_corr_DAs = []
for DA in tqdm(corr_DS.data_vars.values()):
    folded_corr_DAs.append(fold_image(DA, 'chi'))

In [ ]:
selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
                                     'AL31', 'AL41', 'AL28', 'AL38']}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)
len(selected_DAs)

In [ ]:
outPath

In [ ]:
PM6_DS = xr.Dataset()
for DA in selected_DAs:
    PM6_DS[DA.name] = DA.drop_encoding()
    
PM6_DS

In [ ]:
savePath = outPath.joinpath('giwaxs_plots/pm6_data')
savePath.mkdir(exist_ok=True)
PM6_DS.to_zarr(savePath.joinpath('PM6.zarr'), mode='w')

### 2D caked images

#### Folded sin(chi) intensity

In [ ]:
sn

In [ ]:
# Polar plots, for FOLDED sin(chi) intensities
%matplotlib inline
plt.close('all')

# Set chi range: Full range
chi_min = 0
chi_max = 90
q_min = 0.05
q_max = 2.04

# selected_attrs_dict = {}
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
# selected_attrs_dict = {'sample_ID': ['AL22'], 'incident_angle': ['th0.110']}
# selected_attrs_dict = {'sample_ID': ['AL22']}
selected_attrs_dict = {'sample_ID': ['AL38']}

selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)    
for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min, q_max))
    
    # Set color limits
    real_min = float(sliced_DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(sliced_DA.compute().quantile(0.995))       
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(origin='upper', cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(5,4))  # plot, optional parameter interpolation='antialiased' for image smoothing
    # ax.axes.set(title=f'Folded polar plot: {DA.material} {DA.solvent} {DA.rpm}, $\\alpha_i$ = {float(DA.incident_angle[2:])}°, sin($\chi$) corr.')
    ax.axes.set(title=f'Folded polar plot: {sn[DA.sample_ID]}, $\\alpha_i$ = {float(DA.incident_angle[2:])}°, sin($\chi$) corr.')
    ax.colorbar.set_label('Intensity * sin($\chi$) [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    
    # # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    # savePath = outPath.joinpath('giwaxs_plots/caked_plots_v1')
    # savePath.mkdir(exist_ok=True)
    # ax.figure.savefig(savePath.joinpath(f'sinchi-folded_{sn_id[DA.sample_ID]}_{DA.sample_pos}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)

    plt.show()  # Comment to mute plotting output
    # plt.close('all')

### 1D linecuts along chi

In [ ]:
%matplotlib widget
plt.close('all')

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
# chi_min, chi_width, chi_bins = [10, 6, 12]  
chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

q_min = 0.18  # for full qr cut
q_max = 1.9

# Select attribute
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
selected_attrs_dict = {'sample_ID': ['AL22', 'AL25', 'AL26'], 'incident_angle': ['th0.110']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi')

    fig, ax = plt.subplots(figsize=(5.5,3), dpi=150, tight_layout=True)
    for i, chi_bin in enumerate(binned_DA.chi_bins.data[::-1]):
        if i == 0 or i == 17:
            binned_DA.sel(chi_bins=chi_bin).plot.line(ax=ax, color=colors[i], label=chi_bin)

    ax.set(title=f'{sn[DA.sample_ID]} {DA.sample_pos} {DA.incident_angle}: $\\chi$-binned-summed linecuts:\n'+ 
                 f'{chi_bins}, {chi_width}° $\\chi$ bins: from {int(chi_min)} to {int(chi_max)}°', 
           xlabel='$q_r$ $[Å^{-1}]$', 
           ylabel='$sin(\\chi) * Intensity$ [arb. units]')
    ax.axes.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
    # ax.legend(title='Chi Bins')

    # Create a colormap and normalizer to match the plotted data
    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=chi_min, vmax=chi_max))
    sm.set_array([])  # You need to set the array for the scalar mappable

    # Create the colorbar
    cbar = plt.colorbar(sm, ax=ax, orientation='vertical')
    cbar.set_label('Azimuthal Angle [°]', rotation=270, labelpad=15)
    
    # Define the ticks for the colorbar based on the chi bins
    chi_values = np.linspace(chi_min, chi_max, int(chi_bins/2), endpoint=True)
    cbar.set_ticks(chi_values)
    cbar.set_ticklabels([f"{value:.1f}" for value in chi_values])

    # # Save
    # outPath.joinpath('chi-binned_linecuts_v1').mkdir(exist_ok=True)
    # savePath = outPath.joinpath(
    #     'chi-binned_linecuts_v1', f'full_chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
    # # savePath = outPath.joinpath(
    # #     'chi-binned_linecuts_v1', f'pipi_chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
    # savePath.mkdir(exist_ok=True)
    # ax.figure.savefig(savePath.joinpath(f'{DA.film}_{DA.sample_version}_qr{q_min}to{q_max}_{DA.incident_angle}.png'), 
    #                   dpi=150)
    
plt.show()
# plt.close('all')

### 1D line fitting*
Use lmfit to perform the linefits

In [ ]:
# Import relevant lmfit functions:
from lmfit.lineshapes import exponential, linear, pvoigt  # functions that return arrays based on defined function
from lmfit import Model, Parameters, minimize  # lmfit pieces to combose parameters and run minimization fit (can use equivalent Minimizer class instead of minimize)
from lmfit.model import save_modelresult, load_modelresult

In [ ]:
# Define a total function for the curves to be included in the data:

# Total function in this case will have: 1 linear background, 1 exponential decay background and 6 pseudo-voigt peaks:
# Peaks: lamella, backbone, 0p9, 1p2, alkyl, and pipi
def total_func(x, 
               slope_lin, intercept_lin,
               amp_exp, decay_exp,
               amp_lamella, cen_lamella, sigma_lamella, fraction_lamella,
               amp_backbone, cen_backbone, sigma_backbone, fraction_backbone,
               amp_0p9, cen_0p9, sigma_0p9, fraction_0p9,
               amp_1p2, cen_1p2, sigma_1p2, fraction_1p2,
               amp_alkyl, cen_alkyl, sigma_alkyl, fraction_alkyl,
               amp_pipi, cen_pipi, sigma_pipi, fraction_pipi):
    
    return (linear(x, slope_lin, intercept_lin) +
            exponential(x, amp_exp, decay_exp) + 
            pvoigt(x, amp_lamella, cen_lamella, sigma_lamella, fraction_lamella) +
            pvoigt(x, amp_backbone, cen_backbone, sigma_backbone, fraction_backbone) +
            pvoigt(x, amp_0p9, cen_0p9, sigma_0p9, fraction_0p9) +
            pvoigt(x, amp_1p2, cen_1p2, sigma_1p2, fraction_1p2) +
            pvoigt(x, amp_alkyl, cen_alkyl, sigma_alkyl, fraction_alkyl) +
            pvoigt(x, amp_pipi, cen_pipi, sigma_pipi, fraction_pipi))


# Initialize function as an lmfit model object to fit directly
model = Model(total_func, independent_vars=['x'])

# Functions to plot inidividual fit components from parameters later on:
def linear_pars(pars, x):
    slope_lin, intercept_lin = pars[f'slope_lin'].value, pars[f'intercept_lin'].value
    return linear(x, slope_lin, intercept_lin)

def exponential_pars(pars, x):
    amp_exp, decay_exp = pars[f'amp_exp'].value, pars[f'decay_exp'].value
    return exponential(x, amp_exp, decay_exp)

def pvoigt_peak(pars, x, peakname):
    amp, cen, sigma, fraction = pars[f'amp_{peakname}'].value, pars[f'cen_{peakname}'].value, pars[f'sigma_{peakname}'].value, pars[f'fraction_{peakname}'].value
    return pvoigt(x, amp, cen, sigma, fraction)

In [ ]:
# Select data linecut, create parameters, run fit:
# This cell is working for individual linecuts
# Parameters used here are for linefits v2
%matplotlib inline
plt.close('all')

### Select data
# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
# chi_min, chi_width, chi_bins = [10, 6, 12]  
chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

q_min = 0.25  # for full qr cut
q_max = 1.78

# Select attribute
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38'],
#                        'incident_angle':['th0.080', 'th0.110']}
selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26'],
                       'incident_angle':['th0.120', 'th0.150']}

# selected_attrs_dict = {'sample_ID': ['AL38']}
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL25', 'AL26'], 'incident_angle': ['th0.110']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

all_outs = {}
good_ratios = {}
bad_ratios = {}

# Set constant background intercept value location parameters:
bkg_x = 0.56
bkg_extent = 0.015  # how far (plus and minus) around selected point to average 
for bkg_frac in tqdm([0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], desc='background values'):
# for bkg_frac in tqdm([0.6, 0.7, 0.8, 0.9], desc='background values'):
    for DA in tqdm(selected_DAs, desc='samples'):
        sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
        # binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi')
        binned_DA = sliced_DA.groupby_bins('chi', chi_bins).integrate('chi')

        out = None  # clear any default parameters from previous sample

        # Set path to save plots and lmfit results
        fitsPath = outPath.joinpath('full_linefits_v2')
        bkgFracPath = fitsPath.joinpath(f'background-q-{bkg_x}_q-extent-{bkg_extent}_intercept-val-frac-{bkg_frac}')
        bkgFracPath.mkdir(exist_ok=True)
        samplePath = bkgFracPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}')
        samplePath.mkdir(exist_ok=True)

        # for i, chi_bin in tqdm(enumerate(binned_DA.chi_bins.data), total=len(binned_DA.chi_bins.data), desc='chi_bins'):
        for i, chi_bin in enumerate(binned_DA.chi_bins.data):
            chi_key = f'{round(chi_bin.left)}-{round(chi_bin.right)}'

            sel_DA = binned_DA.sel(chi_bins=chi_bin)  

            x = sel_DA.qr.data
            data = sel_DA.data


            bkg_val = float(sel_DA.sel(qr=slice(bkg_x-0.02, bkg_x+0.02)).mean('qr')) * bkg_frac
            # print(f'bkg value: {bkg_val}')

            #  Initialize parameter values & constraints:
            if out and out.redchi<100:  
                # Use previous parameters if they're reasonable (low ish reduced chi value)
                params = out.params
                params.set(intercept_lin = {'value':bkg_val, 'min':bkg_val*0.85, 'max':bkg_val*1.15})
                if chi_bin.left<10:
                    params.set(amp_exp = {'value':0, 'vary':False})
                if chi_bin.left<50:
                    params.set(amp_backbone = {'value':0, 'vary':False})
            else:   
                # Otherwise, initialize new parameters 
                params = Parameters()   
                params.add(f'slope_lin',         value=0,     vary=False)
                params.add(f'intercept_lin',     value=bkg_val, min=bkg_val*0.9, max=bkg_val*1.1)
                if chi_bin.left<10:
                    params.add(f'amp_exp',           value=10,      min=0.01,  max=1e2, vary=False)
                    params.add(f'decay_exp',         value=0.1,     min=0.01,  max=1e2, vary=False)
                else:
                    params.add(f'amp_exp',           value=0,     vary=False)
                    params.add(f'decay_exp',         value=0,     vary=False)
                params.add(f'amp_lamella',       value=250,   min=0,     max=1e4)
                params.add(f'cen_lamella',       value=0.31,  min=0.28,  max=0.34)
                params.add(f'sigma_lamella',     value=0.05,  min=0.01,  max=0.2)
                params.add(f'fraction_lamella',  value=0.5,   min=0,     max=1)
                if chi_bin.left<50:
                    params.add(f'amp_backbone',      value=0, vary=False)
                else:
                    params.add(f'amp_backbone',      value=0.005, min=0,     max=10)
                params.add(f'cen_backbone',      value=0.66,  min=0.63,  max=0.7)
                params.add(f'sigma_backbone',    value=1e-3,  min=0.01,  max=0.05)
                params.add(f'fraction_backbone', value=0.5,   min=0,     max=1)
                params.add(f'amp_0p9',           value=1,     min=0,     max=10)
                params.add(f'cen_0p9',           value=0.93,  min=0.85,  max=0.99)
                params.add(f'sigma_0p9',         value=0.02,  min=0.01,  max=0.4)
                params.add(f'fraction_0p9',      value=0.5,   min=0,     max=1)
                params.add(f'amp_1p2',           value=1,     min=0,     max=10)
                params.add(f'cen_1p2',           value=1.2,   min=1.15,  max=1.25)
                params.add(f'sigma_1p2',         value=0.5,   min=0.05,  max=0.5)
                params.add(f'fraction_1p2',      value=0.5,   min=0,     max=1)
                params.add(f'amp_alkyl',         value=50,    min=0,     max=400)
                params.add(f'cen_alkyl',         value=1.4,   min=1.35,  max=1.45)
                params.add(f'sigma_alkyl',       value=0.5,   min=0.1,   max=0.5)
                params.add(f'fraction_alkyl',    value=0.5,   min=0,     max=1)
                params.add(f'amp_pipi',          value=100,   min=0,     max=1000)
                params.add(f'cen_pipi',          value=1.73,  min=1.65,  max=1.8)
                params.add(f'sigma_pipi',        value=0.1,   min=0.05,  max=0.3)
                params.add(f'fraction_pipi',     value=0.5,   min=0,     max=1)    

            ### Run minimization / fit:
            out = model.fit(data, params, x=x)

            # Retry up to some limit number of times for better fit result
            limit = 10
            counter = 0
            while out.redchi>50:
                # bad fit, retry
                out = model.fit(data, out.params, x=x)
                counter += 1
                if counter >= limit:
                    break            

            # print(f'Retried fit {counter} times...')


            # # Save lmfit model result:
            # save_modelresult(out, samplePath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_{chi_key}.json'))

            # Plot:
            fig, ax = plt.subplots()

            lin_bkg = linear_pars(out.params, x)
            exp_bkg = exponential_pars(out.params, x)
            pvoigt1 = pvoigt_peak(out.params, x, 'lamella')
            pvoigt2 = pvoigt_peak(out.params, x, 'backbone')
            pvoigt3 = pvoigt_peak(out.params, x, '0p9')
            pvoigt4 = pvoigt_peak(out.params, x, '1p2')
            pvoigt5 = pvoigt_peak(out.params, x, 'alkyl')
            pvoigt6 = pvoigt_peak(out.params, x, 'pipi')

            ax.plot(x, data, 'o')
            ax.plot(x, out.best_fit, '-', label='full_fit')
            ax.plot(x, lin_bkg, label=f'lin_bkg')
            ax.plot(x, exp_bkg, label=f'exp_bkg')
            ax.plot(x, pvoigt1, label=f'lamella')
            ax.plot(x, pvoigt2, label=f'backbone')
            ax.plot(x, pvoigt3, label=f'0p9')
            ax.plot(x, pvoigt4, label=f'1p2')
            ax.plot(x, pvoigt5, label=f'alkyl')
            ax.plot(x, pvoigt6, label=f'pipi')
            ax.set_title(f'{sn[sel_DA.sample_ID]} chi bin: {chi_bin}')
            ax.legend(title=f'redchi={np.round(out.redchi, 1)}, abs_resid_mean={np.round(np.abs(out.residual).mean(), 1)}, resid_mean={np.round(out.residual.mean(), 1)}')

            # fig.savefig(samplePath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_{chi_key}.png'))

            plt.show()
            plt.close('all')

In [ ]:
film

In [ ]:
# Plot pseudovoigt fractions, d-spacings, scherrer coherence lengths, and normalized peak areas vs chi
# For each background default value
%matplotlib inline
plt.close('all')

fitsPath = outPath.joinpath('full_linefits_v2')
bkg_all_params = {}
bkg_extracted_value_arrays = {}
for bkgFracPath in tqdm(sorted(fitsPath.glob('background*extent-0.015*'))):
# for bkgFracPath in tqdm(sorted(fitsPath.glob('background*'))):
    # Extract background fraction value from folder name 
    bkg_frac = bkgFracPath.name.split('_')[-1].split('-')[-1]
    # print(bkg_frac)
    
    # Get parameters from saved fit results
    all_params = {}
    for measurementPath in sorted(bkgFracPath.glob('PM6*')):
        measurement_name = measurementPath.name  # get fit name
        
        # Load lmfit results
        params = {}
        for saved_result in sorted(measurementPath.glob('*.json')):
            result_name = saved_result.stem  # stem doesn't include filetype format at end
            chi_bin_key = result_name.split('_')[-1]

            result = load_modelresult(saved_result)

            #only load for chi bins with good fit parameters (low reduced chi and residual average close to zero)
            # if result.redchi<1000 and np.abs(result.residual.mean())<1:
            # if result.redchi<3000:
            params[chi_bin_key] = result.params.valuesdict()

        all_params[measurement_name] = params
        
    bkg_all_params[bkg_frac] = all_params

    # # Choose sample sets (by folder/sample name) (this should probably be outside this loop):
    # peak = 'lamella'
    
    
    # incident_angle = 'th0.110'
    # PM6_CF_samples = [f'PM6_0CN-CF_x0.000_{incident_angle}', f'PM6_p5CN-CF_x0.000_{incident_angle}', f'PM6_1CN-CF_x2.000_{incident_angle}', f'PM6_5CN-CF_x2.000_{incident_angle}']
    # PM6_CB_samples = [f'PM6_0CN-CB_x0.000_{incident_angle}', f'PM6_p5CN-CB_x2.000_{incident_angle}', f'PM6_1CN-CB_x0.000_{incident_angle}', f'PM6_5CN-CB_x0.000_{incident_angle}']


    # Define custom colors for each film, here I've chosen just two shades along the same sequention colorbar for each sample group
    # colors = plt.cm.viridis_r(np.linspace(0,1,len(PM6_CF_samples)))
    # colors = plt.cm.viridis_r(np.linspace(0,1,4))
    # colors = plt.cm.viridis_r(np.linspace(0,1,3))

    # ### Extracting info from all_params & populating ___ vs chi plots:
    # summed_peak_areas = {}
    # avg_dspacings = {}
    # avg_peak_centers = {}
    # avg_coherence_lengths = {}
    
    extracted_value_arrays = {} 
    for i, film in enumerate(all_params.keys()):
    # for i, film in enumerate(PM6_CF_samples):
    # for i, film in enumerate(PM6_CB_samples):
        extracted_arrays = {}
        for peak in ['lamella', 'pipi']:
            peak_areas = []
            pvoigt_fracs = []
            peak_centers = []
            peak_fwhms = []

            chis = []
            for chi_bin, params in all_params[film].items():   
                # Get starting value of chi bin
                chi = int(chi_bin.split('-')[0]) + 2  # set to midpoint of values
                chis.append(chi)
                # Get peak areas
                peak_area = params[f'amp_{peak}']
                peak_areas.append(peak_area)
                # Get pseudovoigt fraction
                pvoigt_frac = params[f'fraction_{peak}']
                pvoigt_fracs.append(pvoigt_frac)
                # Get peak centers
                peak_center = params[f'cen_{peak}']
                peak_centers.append(peak_center)
                # Get peak fwhms
                peak_fwhm = params[f'sigma_{peak}'] * 2
                peak_fwhms.append(peak_fwhm)

            # Get normalized peak areas (divide by sum of all peak areas; adds to 1) and write area sum to dict
            peak_areas = np.array(peak_areas)
            peak_sum = np.sum(peak_areas)
            # summed_peak_areas[film] = np.sum(peak_areas)  # Write sum of peak area out, this corresponds to relative extent of crystallinity if thickness normalized
            normed_peak_areas = peak_areas / np.sum(peak_areas)
            
            extracted_arrays[f'{peak}_peak_sum'] = peak_sum
            extracted_arrays[f'{peak}_normed_peak_areas'] = normed_peak_areas 

            # Calculate d-spacings from peak centers, calculate peak-area-weighted average of d-spacing
            dspacings = (2*np.pi) / np.array(peak_centers) 
            # avg_peak_centers[film] = peak_center
            avg_dspacing = np.round(np.sum(dspacings*normed_peak_areas), 2)
            # avg_dspacings[film] = avg_dspacing  

            extracted_arrays[f'{peak}_dspacings'] = dspacings 


            # Calculate coherence length from peak fwhm, calculate peak-area-weighted average of coherence length    
            coherence_lengths = (2*np.pi*0.9) / np.array(peak_fwhms) 
            avg_coherence_length = np.round(np.sum(coherence_lengths*normed_peak_areas), 2)
            # avg_coherence_lengths[film] = avg_coherence_length

            extracted_arrays[f'{peak}_coherence_lengths'] = coherence_lengths 


        extracted_value_arrays[film] = extracted_arrays

    bkg_extracted_value_arrays[bkg_frac] = extracted_value_arrays 

In [ ]:
bkg_extracted_value_arrays

In [ ]:
# def average_and_error_bars_with_padding(nested_dict):
#     # Initialize dictionaries to hold results
#     average_dict = {}
#     stddev_dict = {}

#     # Loop over measurement names
#     for measurement_name in next(iter(nested_dict.values())):
#         average_dict[measurement_name] = {}
#         stddev_dict[measurement_name] = {}
        
#         # Define array dictionary with keys for each parameter
#         arrays_dict = {
#             'lamella_normed_peak_areas': [], 'lamella_dspacings': [], 'lamella_coherence_lengths': [],
#             'pipi_normed_peak_areas': [], 'pipi_dspacings': [], 'pipi_coherence_lengths': []
#         }

#         # Gather arrays for each parameter across background fractions
#         for background_fraction in nested_dict.keys():
#             for param in arrays_dict:
#                 # Append arrays for this background fraction
#                 arrays_dict[param].append(nested_dict[background_fraction][measurement_name][param])

#         # Process each parameter
#         for param, array_list in arrays_dict.items():
#             # Get the maximum length across all arrays for this parameter
#             max_length = max(arr.shape[0] for arr in array_list)

#             # Pad each array in array_list to the maximum length with NaN
#             padded_arrays = [np.pad(arr, (0, max_length - arr.shape[0]), constant_values=np.nan) for arr in array_list]
            
#             # Stack padded arrays along a new dimension and calculate mean/std deviation while ignoring NaN
#             stacked_array = np.stack(padded_arrays, axis=0)
#             average_dict[measurement_name][param] = np.nanmean(stacked_array, axis=0)
#             stddev_dict[measurement_name][param] = np.nanstd(stacked_array, axis=0)
            
#         # Now also process the peak sum for rDoC :)
        

#     return average_dict, stddev_dict

# # Example usage
# avg_values, err_values = average_and_error_bars_with_padding(bkg_extracted_value_arrays)

In [ ]:
avg_values

In [ ]:
[arr.shape for arr in bkg_extracted_value_arrays['0.3']['PM6_5CN-CF_x0.000_th0.110'].values()]

In [ ]:
[arr.shape for arr in avg_values['PM6_5CN-CF_x0.000_th0.110'].values()]

In [ ]:
def average_and_error_bars_for_arrays(nested_dict):
    # Initialize dictionaries to hold results
    average_dict = {}
    stddev_dict = {}

    # Loop over measurement names
    for measurement_name in next(iter(nested_dict.values())):
        average_dict[measurement_name] = {}
        stddev_dict[measurement_name] = {}
        
        # Gather arrays for each parameter across background fractions
        # arrays_dict = {'normed_peak_areas': [], 'dspacings': [], 'coherence_lengths': []}
        arrays_dict = {'lamella_normed_peak_areas': [], 'lamella_dspacings': [], 'lamella_coherence_lengths': [],
                       'pipi_normed_peak_areas': [], 'pipi_dspacings': [], 'pipi_coherence_lengths': []}

        for background_fraction in nested_dict.keys():
            for param in arrays_dict:
                # Append arrays for this background fraction
                arrays_dict[param].append(nested_dict[background_fraction][measurement_name][param])

        # Stack arrays along the new axis to calculate mean and std deviation
        for param, array_list in arrays_dict.items():
            stacked_array = np.stack(array_list, axis=0)  # Stack along new dimension (background fractions)
            average_dict[measurement_name][param] = np.mean(stacked_array, axis=0)  # Average along background fraction axis
            stddev_dict[measurement_name][param] = np.std(stacked_array, axis=0)  # Stddev along background fraction axis
            
        # Now also do for peak sum (-> rDoC)
        sum_arrays_dict = {'lamella_peak_sum': [], 'pipi_peak_sum': []}
        for background_fraction in nested_dict.keys():
            for param in sum_arrays_dict:
                # Append arrays for this background fraction
                sum_arrays_dict[param].append(nested_dict[background_fraction][measurement_name][param])

        # Stack arrays along the new axis to calculate mean and std deviation
        for param, sum_array_list in sum_arrays_dict.items():
            stacked_array = np.stack(sum_array_list, axis=0)  # Stack along new dimension (background fractions)
            average_dict[measurement_name][param] = np.mean(stacked_array, axis=0)  # Average along background fraction axis
            stddev_dict[measurement_name][param] = np.std(stacked_array, axis=0)  # Stddev along background fraction axis        

    return average_dict, stddev_dict

avg_values, err_values = average_and_error_bars_for_arrays(bkg_extracted_value_arrays)


In [ ]:
avg_values

In [ ]:
err_values

In [ ]:
# def average_and_error_bars(nested_dict):
#     # Initialize dictionaries to hold results
#     average_dict = {}
#     stddev_dict = {}

#     # Loop over measurement names
#     for measurement_name, chi_bins in next(iter(nested_dict.values())).items():
#         average_dict[measurement_name] = {}
#         stddev_dict[measurement_name] = {}

#         # Loop over chi bins
#         for chi_bin, _ in chi_bins.items():
#             # Gather parameter values for each background fraction
#             params_dict = {}
#             for background_fraction in nested_dict:
#                 fit_results = nested_dict[background_fraction][measurement_name][chi_bin]
#                 for param, value in fit_results.items():
#                     if param not in params_dict:
#                         params_dict[param] = []
#                     params_dict[param].append(value)

#             # Compute average and standard deviation for each parameter
#             average_dict[measurement_name][chi_bin] = {}
#             stddev_dict[measurement_name][chi_bin] = {}
#             for param, values in params_dict.items():
#                 average_dict[measurement_name][chi_bin][param] = np.mean(values)
#                 stddev_dict[measurement_name][chi_bin][param] = np.std(values)

#     return average_dict, stddev_dict

# # Example usage
# avg_params, err_params = average_and_error_bars(bkg_all_params)

# # Output the results
# print("Averages:\n", avg)
# print("\nStandard Deviations (Error Bars):\n", err)


### Here

In [ ]:
plt.cm.Greys

In [ ]:
%matplotlib inline
plt.close('all')

# peak = 'pipi'
for peak in ['lamella', 'pipi']:
# for peak in ['pipi']:
    for solvent in ['CF', 'CB']:
    # for solvent in ['CB']:  # dont yet have .120 and .150 CB
        avgs_dict = {}
        errs_dict = {}
        for incident_angle in ['th0.080', 'th0.110', 'th0.120', 'th0.150']:
            # # Initialize figures to be populated later
            # fig_dspacing, ax_dspacing = plt.subplots(figsize=((4.5,2.5)), dpi=150, constrained_layout=True)
            # fig_ccl, ax_ccl = plt.subplots(figsize=((4.5,2.5)), dpi=150, constrained_layout=True)
            # fig_pole, ax_pole = plt.subplots(figsize=((4.5,2.5)), dpi=150, constrained_layout=True)
            

            alphai = float(incident_angle[2:])
            alpha = '$α_{inc}$'

            if solvent == 'CF':
                PM6_samples = [f'PM6_0CN-CF_x0.000_{incident_angle}', f'PM6_p5CN-CF_x0.000_{incident_angle}', f'PM6_1CN-CF_x2.000_{incident_angle}', f'PM6_5CN-CF_x2.000_{incident_angle}']
            elif solvent == 'CB':
                PM6_samples = [f'PM6_0CN-CB_x0.000_{incident_angle}', f'PM6_p5CN-CB_x2.000_{incident_angle}', f'PM6_1CN-CB_x0.000_{incident_angle}', f'PM6_5CN-CB_x0.000_{incident_angle}']

            additive_label = ['0.0% CN', '0.5% CN', '1.0% CN', '5.0% CN']

            # Define custom colors for each film, here I've chosen just two shades along the same sequention colorbar for each sample group
            # colors = plt.cm.viridis_r(np.linspace(0,1,len(PM6_CF_samples)))
            colors = plt.cm.viridis_r(np.linspace(0,1,4))
            # Define custom colors for each parameter to plot in line plot summary
            param_colors = plt.cm.plasma(np.linspace(0, 0.9, 3))  # orientation, ccl, d-spacing

            ### Extracting info from all_params & populating ___ vs chi plots:
            peak_sums = []
            peak_sum_errs = []
            avg_chis = []
            avg_Ss = []
            avg_ccls = []
            avg_dspacings = []
            avg_chi_errs = []
            avg_ccl_errs = []
            avg_dspacing_errs = []
            # for i, film in enumerate(all_params.keys()):

            chis = np.array([12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80])

            for i, film in enumerate(PM6_samples):

                # Get normalized peak areas from averaged dictionary
                normed_peak_areas = avg_values[film][f'{peak}_normed_peak_areas']
                normed_peak_errs = err_values[film][f'{peak}_normed_peak_areas']
                avg_chi = np.round((normed_peak_areas*chis).sum(), 2)
                avg_fz = np.round((normed_peak_areas*np.cos(np.deg2rad(chis))**2).sum(), 2)
                # avg_fz = np.round((normed_peak_areas*np.deg2rad(chis)*np.cos(np.deg2rad(chis))**2).sum(), 2)
                avg_S = (1/2) * ((3 * avg_fz) - 1)
                avg_Ss.append(avg_S)
                # avg_chi_err = np.round((normed_peak_errs*chis).mean(), 2)  # this should actually be mean() instead of sum?
                avg_chi_err = np.round(np.sqrt(np.sum((chis-avg_chi)**2 * normed_peak_errs**2)), 2)  # error propogation from GPT
                avg_chis.append(avg_chi)
                avg_chi_errs.append(avg_chi_err)
                
                # Get peak sum:
                peak_sum = avg_values[film][f'{peak}_peak_sum']
                peak_sum_err = err_values[film][f'{peak}_peak_sum']
                peak_sums.append(peak_sum)
                peak_sum_errs.append(peak_sum_err)
            
                # Get coherence lengths from averaged dictionary
                coherence_lengths = avg_values[film][f'{peak}_coherence_lengths']
                coherence_length_errs = err_values[film][f'{peak}_coherence_lengths']
                avg_ccl = np.round((coherence_lengths*normed_peak_areas).sum(), 2)  
                avg_ccl_err = np.round((coherence_length_errs*normed_peak_areas).sum(), 4)
                # avg_ccl = np.round(coherence_lengths.mean(), 2)
                # avg_ccl_err = np.round(coherence_length_errs.mean(), 2)
                avg_ccls.append(avg_ccl)
                avg_ccl_errs.append(avg_ccl_err)
                
                # Get dspacings from averaged dictionary
                dspacings = avg_values[film][f'{peak}_dspacings']
                dspacing_errs = err_values[film][f'{peak}_dspacings']
                avg_dspacing = np.round((dspacings*normed_peak_areas).sum(), 2)
                avg_dspacing_err = np.round((dspacing_errs*normed_peak_areas).sum(), 2)
                # avg_dspacing = np.round(dspacings.mean(), 2)
                # avg_dspacing_err = np.round(dspacing_errs.mean(), 2)
                avg_dspacings.append(avg_dspacing)
                avg_dspacing_errs.append(avg_dspacing_err)
                                
                ## Plotting
                # Plot d-spacing vs chi for each film
                ax_dspacing.errorbar(chis, dspacings, label=f'{additive_label[i]}: {avg_dspacing} ± {avg_dspacing_err}', color=colors[i], marker='.', 
                                     capsize=4, yerr=dspacing_errs)
                ax_dspacing.set(title=f'PM6 {solvent}, {alpha}={alphai}°: {peak} d-spacing vs $\\chi$',
                                ylabel='d-spacing [Å]',
                                xlabel='$\\chi$ value [°]')
                                # ylim=(3.55,3.8))
                ax_dspacing.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.9))
                ax_dspacing.grid(visible=True,which='major',axis='y')



                # Plot coherence length vs chi for each film
                if peak == 'pipi' and additive_label[i]=='5.0% CN' and False:
                    ax_ccl.errorbar(chis[:10], coherence_lengths[:10], label=f'{additive_label[i]}: {avg_ccl} ± {avg_ccl_err}', color=colors[i], marker='.', 
                                    capsize=4, yerr=coherence_length_errs[:10])
                    ax_ccl.set(title=f'PM6 {solvent}, {alpha}={alphai}°: {peak} CCL vs $\\chi$',
                               ylabel='CCL [Å]',
                               xlabel='$\\chi$ value [°]')
                               # ylim=(13,18.5))
                    ax_ccl.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.9))
                    ax_ccl.grid(visible=True,which='major',axis='y')
                else:
                    ax_ccl.errorbar(chis, coherence_lengths, label=f'{additive_label[i]}: {avg_ccl} ± {avg_ccl_err}', color=colors[i], marker='.', 
                                    capsize=4, yerr=coherence_length_errs)
                    ax_ccl.set(title=f'PM6 {solvent}, {alpha}={alphai}°: {peak} CCL vs $\\chi$',
                               ylabel='CCL [Å]',
                               xlabel='$\\chi$ value [°]')
                               # ylim=(13,18.5))
                    ax_ccl.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.9))
                    ax_ccl.grid(visible=True,which='major',axis='y')
                    
                if peak == 'pipi':
                    ax_ccl.set_ylim((5,35))
                else:
                    ax_ccl.set_ylim((25,175))

                # Plot 'pole figure' for each film
                ax_pole.errorbar(chis, normed_peak_areas, label=f'{additive_label[i]}: {avg_chi} ± {avg_chi_err}', color=colors[i], marker='.',
                                 capsize=4, yerr=normed_peak_errs)
                ax_pole.set(title=f'PM6 {solvent}, {alpha}={alphai}°: {peak} pole figure',
                            ylabel='Peak area [arb. units]',
                            xlabel='$\\chi$ value [°]')
                ax_pole.set_ybound(lower=-0.005, upper=0.17)
                ax_pole.grid(visible=True,which='major',axis='y')
                # ax_pole.legend(title='Film')    
                ax_pole.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.9))

                # # Save
                # savePath = outPath.joinpath('giwaxs_plots/linefit_summaries_v3')
                # fig_dspacing.savefig(savePath.joinpath(f'PM6_{solvent}_{peak}_dspacings_{incident_angle}.png'), dpi=120, bbox_inches='tight')
                # fig_ccl.savefig(savePath.joinpath(f'PM6_{solvent}_{peak}_coherence_lengths_{incident_angle}.png'), dpi=120, bbox_inches='tight')
                # fig_pole.savefig(savePath.joinpath(f'PM6_{solvent}_{peak}_peak_areas_{incident_angle}.png'), dpi=120, bbox_inches='tight')

            plt.show()
                
            avgs_dict[f'{incident_angle}_chis'] = avg_chis
            # avgs_dict[f'{incident_angle}_Ss'] = avg_Ss
            avgs_dict[f'{incident_angle}_ccls'] = avg_ccls
            avgs_dict[f'{incident_angle}_dspacings'] = avg_dspacings
            avgs_dict[f'{incident_angle}_sums'] = peak_sums
            errs_dict[f'{incident_angle}_chi_errs'] = avg_chi_errs
            errs_dict[f'{incident_angle}_ccl_errs'] = avg_ccl_errs
            errs_dict[f'{incident_angle}_dspacing_errs'] = avg_dspacing_errs
            errs_dict[f'{incident_angle}_sum_errs'] = peak_sum_errs
            
            fig_summary, ax1_summary = plt.subplots(figsize=(5.5,2.5), dpi=150, tight_layout=False)
            ax1_summary.errorbar([label.split()[0][:-1] for label in additive_label], 
                                 avg_dspacings, 
                                 yerr=avg_dspacing_errs, 
                                 color=param_colors[2],
                                 marker='o',
                                 capsize=4,
                                 label='d-spacing') 
            ax1_summary.set_ylabel('d-spacing [Å]')
            if peak == 'lamella':
                ax1_summary.set_ylim((18.5,21.5))
            else:
            # if peak == 'pipi':
                ax1_summary.set_ylim((3.5,3.7))

            ax2_summary = ax1_summary.twinx()
            ax2_summary.errorbar([label.split()[0][:-1] for label in additive_label], 
                                 avg_ccls, 
                                 yerr=avg_ccl_errs, 
                                 color=param_colors[1],
                                 marker='o',
                                 capsize=4,
                                 label='CCL')
            ax2_summary.set_ylabel('CCL [Å]')
            if peak == 'lamella':
                ax2_summary.set_ylim((20,130))
            else:
            # if peak == 'pipi':
                ax2_summary.set_ylim((5,30))
                
            
                
            ax3_summary = ax1_summary.twinx()
            ax3_summary.spines['right'].set_position(('outward', 45))  # Offset the third axis
            ax3_summary.errorbar([label.split()[0][:-1] for label in additive_label], 
                                 avg_chis, 
                                 yerr=avg_chi_errs, 
                                 color=param_colors[0],
                                 marker='o',
                                 capsize=4,
                                 label='Stacking angle')
            ax3_summary.set_ylabel('Stacking angle χ [°]')
            ax3_summary.set_xlabel('CN additive [vol %]')
            ax3_summary.set_ylim((20,70))

            ax4_summary = ax1_summary.twinx()
            ax4_summary.spines['right'].set_position(('outward', 90))  # Offset the third axis
            ax4_summary.errorbar([label.split()[0][:-1] for label in additive_label], 
                                 avg_Ss, 
                                 # yerr=avg_chi_errs, 
                                 color='orange',
                                 marker='o',
                                 capsize=4,
                                 label='S')
            ax4_summary.set_ylabel('Uniaxial order parameter S')
            ax4_summary.set_xlabel('CN additive [vol %]')
            ax4_summary.set_ylim((-0.5,1))

            fig_summary.legend(loc='upper right', bbox_to_anchor=[1.45,0.62])
            fig_summary.suptitle(f'PM6 {solvent}, {alpha}={alphai}°\n Average {peak} fit parameters', y=0.9)

            # # Save
            # savePath = outPath.joinpath('giwaxs_plots/linefit_summaries_v3')
            # fig_summary.savefig(savePath.joinpath(f'PM6_{solvent}_{peak}_{incident_angle}_summary.png'), dpi=150, bbox_inches='tight')

            
            plt.show()
            plt.close('all')  

#         # display(avgs_dict)
#         # display(errs_dict)
        
        aoi_colors = plt.cm.winter_r(np.linspace(0,1,4))
        # aoi_colors = plt.cm.turbo(np.linspace(0,1,4))
        colors_dict = {
            'th0.080': aoi_colors[0],
            'th0.110': aoi_colors[1],
            'th0.120': aoi_colors[2],
            'th0.150': aoi_colors[3],
        }
        sums_colors = plt.cm.Greys(np.linspace(0.35,1,4))
        sums_colors_dict = {
            'th0.080': sums_colors[0],
            'th0.110': sums_colors[1],
            'th0.120': sums_colors[2],
            'th0.150': sums_colors[3],
        }
        linestyles_dict = {
            'ccls': ':',
            'chis': '-',
            'sums': '--'
        }
            
                
        fig_aois, ax1_aois = plt.subplots(figsize=(2,2), dpi=150)
        ax2_aois = None
        ax3_aois = None
        for angle_ptype, avgs, errs in zip(avgs_dict.keys(), avgs_dict.values(), errs_dict.values()):
            aoi = angle_ptype.split('_')[0]
            ptype = angle_ptype.split('_')[1]
            
            alphai = float(aoi[2:])
            alpha = '$α_{inc}$'
                        
            if ptype == 'chis':
                ax1_aois.errorbar([label.split()[0][:-1] for label in additive_label], 
                                     avgs, 
                                     yerr=errs, 
                                     color=colors_dict[aoi],
                                     marker='.',
                                     linestyle=linestyles_dict[ptype],
                                     capsize=2,
                                     label=f'Stacking angle: {alpha}={alphai}°')
                ax1_aois.set_ylabel('Stacking angle χ [°]')
                ax1_aois.set_xlabel('CN additive [vol %]')

            if peak == 'lamella' and ptype == 'ccls':
            # if ptype == 'ccls':            
                if ax2_aois is None:
                    ax2_aois = ax1_aois.twinx()

                ax2_aois.errorbar([label.split()[0][:-1] for label in additive_label], 
                                     avgs, 
                                     yerr=errs, 
                                     color=colors_dict[aoi],
                                     linestyle=linestyles_dict[ptype],
                                     marker='.',
                                     capsize=2,
                                     label=f'CCL: {alpha}={alphai}°')
                ax2_aois.set_ylabel('CCL [Å]')
            elif peak == 'pipi' and ptype == 'ccls':
                if ax2_aois is None:
                    ax2_aois = ax1_aois.twinx()

                ax2_aois.errorbar([label.split()[0][:-1] for label in additive_label], 
                                     avgs, 
                                     yerr=errs, 
                                     color=colors_dict[aoi],
                                     linestyle=linestyles_dict[ptype],
                                     marker='.',
                                     capsize=2,
                                     label=f'CCL: {alpha}={alphai}°')
                ax2_aois.set_ylabel('CCL [Å]')  
                ax2_aois.set_ylim((5,40))
                
#             if ptype == 'sums' and aoi == 'th0.080':
#             # if ptype == 'sums':
#                 if ax3_aois is None:
#                     ax3_aois = ax1_aois.twinx()
#                     ax3_aois.spines['right'].set_position(('outward', 45))  # Offset the third axis
                    
#                 ax3_aois.errorbar([label.split()[0][:-1] for label in additive_label], 
#                                      avgs, 
#                                      yerr=errs, 
#                                      color=sums_colors_dict[aoi],
#                                      linestyle=linestyles_dict[ptype],
#                                      marker='.',
#                                      capsize=2,
#                                      label=f'rDoC: {alpha}={alphai}°')
#                 ax3_aois.set_ylabel('Peak sum [a.u.]')  
            
            

        fig_aois.legend(loc='upper right', bbox_to_anchor=[3,0.95])
        fig_aois.suptitle(f'PM6 {solvent} average {peak} fit parameters', y=0.98)

        # # Save
        # savePath = outPath.joinpath('giwaxs_plots/linefit_summaries_v3')
        # fig_aois.savefig(savePath.joinpath(f'PM6_{solvent}_{peak}_aois.png'), dpi=150, bbox_inches='tight')


        plt.show()
        plt.close('all')  
            


In [ ]:
avg_chis

In [ ]:
0.5*(3*np.cos(np.deg2rad(avg_chis))**2 - 1)

In [ ]:
colors_dict = {
    
}
linestyles_dict = {}
        fig_aois, ax1_aois = plt.subplots(figsize=(3.5,2.5), dpi=150, tight_layout=True)
        ax1_aois.errorbar([label.split()[0][:-1] for label in additive_label], 
                             avg_chis, 
                             yerr=avg_chi_errs, 
                             color=param_colors[0],
                             marker='o',
                             capsize=4,
                             label='Stacking angle')
        ax1_aois.set_ylabel('Stacking angle χ [°]')
        ax1_aois.set_xlabel('CN additive [vol %]')

        if peak == 'lamella':
            ax2_aois = ax1_summary.twinx()
            ax2_aois.errorbar([label.split()[0][:-1] for label in additive_label], 
                                 avg_ccls, 
                                 yerr=avg_ccl_errs, 
                                 color=param_colors[1],
                                 marker='o',
                                 capsize=4,
                                 label='CCL')
            ax2_aois.set_ylabel('CCL [Å]')

        fig_summary.legend(loc='upper right', bbox_to_anchor=[1.45,0.62])
        fig_summary.suptitle(f'PM6 {solvent}, {alpha}={alphai}°\n Average {peak} fit parameters', y=0.9)

        # # Save
        # savePath = outPath.joinpath('giwaxs_plots/linefit_summaries_v1')
        # fig_summary.savefig(savePath.joinpath(f'PM6_{solvent}_{peak}_{incident_angle}_summary.png'), dpi=150, bbox_inches='tight')

In [ ]:
(np.pi*2)/3.55

In [ ]:
# # Load all_params object from saved folders, but only do so if 
# bkg_frac = bkgFracPath.name.split('_')[-1].split('-')[-1]  # extract background fraction from folder name

# all_params = {}
# for measurementPath in bkgFracPath.glob('PM6*'):
#     measurement_name = measurementPath.name  # get fit name
    
#     params = {}
    
#     # Load lmfit results
#     for saved_result in measurementPath.glob('*.json'):
#         result_name = saved_result.stem  # stem doesn't include filetype format at end
#         chi_bin_key = result_name.split('_')[-1]
        
#         result = load_modelresult(saved_result)
        
#         #only load for chi bins with good fit parameters (low reduced chi and residual average close to zero)
#         if result.redchi<100 and np.abs(result.residual.mean())<1:
#             params[chi_bin_key] = result.params.valuesdict()
        
#     all_params[measurement_name] = params
    
# # all_params

In [ ]:
# # Previous code used for fitting peak results extracted from 'all_params' dictionaries
# # The 'all_params dict' goes sample/scan key, chi_bin key, parameters dict

# # Plot pseudovoigt fractions, d-spacings, scherrer coherence lengths, and normalized peak areas vs chi
# %matplotlib inline
# plt.close('all')

# # Initialize figures to be populated later
# fig_psvoigt, ax_psvoigt = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
# fig_dspacing, ax_dspacing = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
# fig_ccl, ax_ccl = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
# fig_pole, ax_pole = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
# # fig_pole, ax_pole = plt.subplots(figsize=((2.8,3.3)), dpi=150, tight_layout=True)

# # # Choose sample sets (by folder/sample name):
# peak = '0p9'
# # incident_angle = 'th0.110'
# # PM6_CF_samples = [f'PM6_0CN-CF_x0.000_{incident_angle}', f'PM6_p5CN-CF_x0.000_{incident_angle}', f'PM6_1CN-CF_x2.000_{incident_angle}', f'PM6_5CN-CF_x2.000_{incident_angle}']
# # PM6_CB_samples = [f'PM6_0CN-CB_x0.000_{incident_angle}', f'PM6_p5CN-CB_x0.000_{incident_angle}', f'PM6_1CN-CB_x0.000_{incident_angle}', f'PM6_5CN-CB_x0.000_{incident_angle}']


# # Define custom colors for each film, here I've chosen just two shades along the same sequention colorbar for each sample group
# # colors = plt.cm.viridis_r(np.linspace(0,1,len(PM6_CF_samples)))
# colors = plt.cm.viridis_r(np.linspace(0,1,3))

# ### Extracting info from all_params & populating ___ vs chi plots:
# summed_peak_areas = {}
# avg_dspacings = {}
# avg_peak_centers = {}
# avg_coherence_lengths = {}
# for i, film in enumerate(all_params.keys()):
# # for i, film in enumerate(PM6_CF_samples):
# # for i, film in enumerate(PM6_CB_samples):
#     peak_areas = []
#     pvoigt_fracs = []
#     peak_centers = []
#     peak_fwhms = []
    
#     chis = []
#     for chi_bin, params in all_params[film].items():   
#         # Get starting value of chi bin
#         chi = int(chi_bin.split('-')[0]) + 2  # set to midpoint of values
#         chis.append(chi)
#         # Get peak areas
#         peak_area = params[f'amp_{peak}']
#         peak_areas.append(peak_area)
#         # Get pseudovoigt fraction
#         pvoigt_frac = params[f'fraction_{peak}']
#         pvoigt_fracs.append(pvoigt_frac)
#         # Get peak centers
#         peak_center = params[f'cen_{peak}']
#         peak_centers.append(peak_center)
#         # Get peak fwhms
#         peak_fwhm = params[f'sigma_{peak}'] * 2
#         peak_fwhms.append(peak_fwhm)

#     # Get normalized peak areas (divide by sum of all peak areas; adds to 1) and write area sum to dict
#     peak_areas = np.array(peak_areas)
#     summed_peak_areas[film] = np.sum(peak_areas)  # Write sum of peak area out, this corresponds to relative extent of crystallinity if thickness normalized
#     normed_peak_areas = peak_areas / np.sum(peak_areas)

#     # Calculate d-spacings from peak centers, calculate peak-area-weighted average of d-spacing
#     dspacings = (2*np.pi) / np.array(peak_centers) 
#     avg_peak_centers[film] = peak_center
#     avg_dspacing = np.round(np.sum(dspacings*normed_peak_areas), 2)
#     avg_dspacings[film] = avg_dspacing  

#     # Calculate coherence length from peak fwhm, calculate peak-area-weighted average of coherence length    
#     coherence_lengths = (2*np.pi*0.9) / np.array(peak_fwhms) 
#     avg_coherence_length = np.round(np.sum(coherence_lengths*normed_peak_areas), 2)
#     avg_coherence_lengths[film] = avg_coherence_length

#     ## Plotting
#     # Plot pseudovoigt fraction vs chi for each film
#     ax_psvoigt.errorbar(chis, pvoigt_fracs, label=film, color=colors[i], marker='o')
#     ax_psvoigt.set(title=f'{peak} pseudo-voigt lorentzian-to-gaussian ratio versus $\\chi$',
#                    ylabel='Lorentzian peak fraction',
#                    xlabel='Binned $\\chi$ value [°]')
#     ax_psvoigt.legend(title='Film', loc='upper left', bbox_to_anchor=(1,1))

#     # Plot d-spacing vs chi for each film
#     ax_dspacing.errorbar(chis, dspacings, label=film, color=colors[i], marker='o', 
#                          capsize=4)
#     ax_dspacing.set(title=f'{peak} d-spacing versus $\\chi$',
#                     ylabel='d-spacing [Å]',
#                     xlabel='$\\chi$ value [°]')
#                     # ylim=(3.55,3.8))
#     ax_dspacing.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))
#     ax_dspacing.grid(visible=True,which='major',axis='y')

#     # Plot coherence length vs chi for each film
#     ax_ccl.errorbar(chis, coherence_lengths, label=film, color=colors[i], marker='o', 
#                     capsize=4)
#     ax_ccl.set(title=f'{peak} crystalline coherence length (CCL) versus $\\chi$',
#                ylabel='CCL [Å]',
#                xlabel='$\\chi$ value [°]')
#                # ylim=(13,18.5))
#     ax_ccl.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))
#     ax_ccl.grid(visible=True,which='major',axis='y')
    
#     # Plot 'pole figure' for each film
#     ax_pole.errorbar(chis, normed_peak_areas, label=film, color=colors[i], marker='o',
#                      capsize=4)
#     ax_pole.set(title=f'{peak} pole figure',
#                 ylabel='Peak area [arb. units]',
#                 xlabel='$\\chi$ value [°]')
#     # ax_pole.set_ybound(lower=0, upper=0.21)
#     ax_pole.grid(visible=True,which='major',axis='y')
#     # ax_pole.legend(title='Film')    
#     ax_pole.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))

#     # Extract chi values for file saving purposes
#     chi_width = chis[0]-chis[1]
#     chi_bins = len(chis)
#     chi_min = chis[-1] - 1
#     chi_max = chis[0]+chi_width - 1

#     # # Save
#     # parent_folder = 'full_chi_fit_results_plots_v1'
#     # outPath.joinpath(parent_folder).mkdir(exist_ok=True)
#     # savePath = outPath.joinpath(parent_folder, f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
#     # savePath.mkdir(exist_ok=True)
#     # fig_psvoigt.savefig(savePath.joinpath(f'pseudovoigt_ratios_pi-pi_fit.png'), dpi=150)
#     # fig_dspacing.savefig(savePath.joinpath(f'dspacings_pi-pi_fit.png'), dpi=150)
#     # fig_ccl.savefig(savePath.joinpath(f'coherence_lengths_pi-pi_fit.png'), dpi=150)
#     # fig_pole.savefig(savePath.joinpath(f'peak_areas_pi-pi_fit.png'), dpi=150)
#     # # fig_pole.savefig(savePath.joinpath(f'peak_areas_pi-pi_fit_narrow.png'), dpi=150)
    
# plt.show()
# plt.close('all')  

#### Unused code for using residual + minimize fitting (same as above but less useful result objects)

In [ ]:
# Functions for if using a residual function and lmfit.minimize 
def linear_pars(pars, x):
    slope_lin, intercept_lin = pars[f'slope_lin'].value, pars[f'intercept_lin'].value
    return linear(x, slope_lin, intercept_lin)

def exponential_pars(pars, x):
    amp_exp, decay_exp = pars[f'amp_exp'].value, pars[f'decay_exp'].value
    return exponential(x, amp_exp, decay_exp)

def pvoigt_peak(pars, x, peakname):
    amp, cen, sigma, fraction = pars[f'amp_{peakname}'].value, pars[f'cen_{peakname}'].value, pars[f'sigma_{peakname}'].value, pars[f'fraction_{peakname}'].value
    return pvoigt(x, amp, cen, sigma, fraction)
        
# Define residiual function to minimize for a fit of an individual linecut:
def residual(pars, x, data):
    model = total_func(x, 
                       pars['slope_lin'].value, pars['intercept_lin'].value,
                       pars['amp_exp'].value, pars['decay_exp'].value,
                       pars['amp_lamella'].value, pars['cen_lamella'].value, pars['sigma_lamella'].value, pars['fraction_lamella'].value,
                       pars['amp_backbone'].value, pars['cen_backbone'].value, pars['sigma_backbone'].value, pars['fraction_backbone'].value,
                       pars['amp_0p9'].value, pars['cen_0p9'].value, pars['sigma_0p9'].value, pars['fraction_0p9'].value,
                       pars['amp_1p2'].value, pars['cen_1p2'].value, pars['sigma_1p2'].value, pars['fraction_1p2'].value,
                       pars['amp_alkyl'].value, pars['cen_alkyl'].value, pars['sigma_alkyl'].value, pars['fraction_alkyl'].value,
                       pars['amp_pipi'].value, pars['cen_pipi'].value, pars['sigma_pipi'].value, pars['fraction_pipi'].value)
    return model - data

In [ ]:
# Select data linecut, create parameters, run fit:
# This cell is working for individual linecuts, using residual function syntax
%matplotlib inline
plt.close('all')

### Select data
# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
# chi_min, chi_width, chi_bins = [10, 6, 12]  
chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

q_min = 0.25  # for full qr cut
q_max = 1.78

# Select attribute
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
selected_attrs_dict = {'sample_ID': ['AL22', 'AL25', 'AL26'], 'incident_angle': ['th0.110']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

all_outs = {}
good_ratios = {}
bad_ratios = {}
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi')
    
    out = None  # clear any default parameters from previous sample
    ratios = {}
    for i, chi_bin in tqdm(enumerate(binned_DA.chi_bins.data), total=len(binned_DA.chi_bins.data)):
        sel_DA = binned_DA.sel(chi_bins=chi_bin)  

        x = sel_DA.qr.data
        data = sel_DA.data
        
        bkg_x = 0.56
        bkg_frac = 0.4
        bkg_val = float(sel_DA.sel(qr=slice(bkg_x-0.01, bkg_x+0.01)).mean('qr')) * bkg_frac
        print(f'bkg value: {bkg_val}')
            
        #  Initialize parameter values & constraints:
        if out and out.redchi<100:
            params = out.params
            params.set(intercept_lin = {'value':bkg_val, 'min':bkg_val*0.85, 'max':bkg_val*1.15})
            if chi_bin.left<10:
                params.set(amp_exp = {'value':0, 'vary':False})
            if chi_bin.left<50:
                params.set(amp_backbone = {'value':0, 'vary':False})
        else:
            params = Parameters()   
            params.add(f'slope_lin',         value=0,     vary=False)
            params.add(f'intercept_lin',     value=bkg_val, min=bkg_val*0.9, max=bkg_val*1.1)
            if chi_bin.left<10:
                params.add(f'amp_exp',           value=10,      min=0.01,  max=1e2, vary=False)
                params.add(f'decay_exp',         value=0.1,     min=0.01,  max=1e2, vary=False)
            else:
                params.add(f'amp_exp',           value=0,     vary=False)
                params.add(f'decay_exp',         value=0,     vary=False)
            params.add(f'amp_lamella',       value=250,   min=0,     max=1e4)
            params.add(f'cen_lamella',       value=0.31,  min=0.28,  max=0.34)
            params.add(f'sigma_lamella',     value=0.05,  min=0.01,  max=0.2)
            params.add(f'fraction_lamella',  value=0.5,   min=0,     max=1)
            if chi_bin.left<50:
                params.add(f'amp_backbone',      value=0, vary=False)
            else:
                params.add(f'amp_backbone',      value=0.005, min=0,     max=10)
            params.add(f'cen_backbone',      value=0.66,  min=0.63,  max=0.7)
            params.add(f'sigma_backbone',    value=1e-3,  min=0.01,  max=0.05)
            params.add(f'fraction_backbone', value=0.5,   min=0,     max=1)
            params.add(f'amp_0p9',           value=1,     min=0,     max=10)
            params.add(f'cen_0p9',           value=0.93,  min=0.85,  max=0.99)
            params.add(f'sigma_0p9',         value=0.02,  min=0.01,  max=0.4)
            params.add(f'fraction_0p9',      value=0.5,   min=0,     max=1)
            params.add(f'amp_1p2',           value=1,     min=0,     max=10)
            params.add(f'cen_1p2',           value=1.2,   min=1.15,  max=1.25)
            params.add(f'sigma_1p2',         value=0.5,   min=0.05,  max=0.5)
            params.add(f'fraction_1p2',      value=0.5,   min=0,     max=1)
            params.add(f'amp_alkyl',         value=50,    min=0,     max=400)
            params.add(f'cen_alkyl',         value=1.4,   min=1.35,  max=1.45)
            params.add(f'sigma_alkyl',       value=0.5,   min=0.1,   max=0.5)
            params.add(f'fraction_alkyl',    value=0.5,   min=0,     max=1)
            params.add(f'amp_pipi',          value=100,   min=0,     max=1000)
            params.add(f'cen_pipi',          value=1.73,  min=1.65,  max=1.8)
            params.add(f'sigma_pipi',        value=0.1,   min=0.05,  max=0.3)
            params.add(f'fraction_pipi',     value=0.5,   min=0,     max=1)    

        ### Run minimization(s):
        out = minimize(residual, params, args=(x,data))
        
        # Retry up to some limit number of times for better fit result
        limit = 10
        counter = 0
        while out.redchi>50:
            # bad fit, retry
            out = minimize(residual, out.params, args=(x,data))
            counter += 1
            if counter >= limit:
                break            
        
        print(f'Retried fit {counter} times...')
                
        full_fit = data + out.residual
        
        all_outs[f'{sn_id[binned_DA.sample_ID]}_{binned_DA.incident_angle}'] = out


        # Plot:
        fig, ax = plt.subplots()
        lin_bkg = linear_pars(out.params, x)
        exp_bkg = exponential_pars(out.params, x)
        pvoigt1 = pvoigt_peak(out.params, x, 'lamella')
        pvoigt2 = pvoigt_peak(out.params, x, 'backbone')
        pvoigt3 = pvoigt_peak(out.params, x, '0p9')
        pvoigt4 = pvoigt_peak(out.params, x, '1p2')
        pvoigt5 = pvoigt_peak(out.params, x, 'alkyl')
        pvoigt6 = pvoigt_peak(out.params, x, 'pipi')

        sig_lamella = out.params['sigma_lamella'].value
        sig_backbone = out.params['sigma_backbone'].value
        sig_0p9 = out.params['sigma_0p9'].value
        sig_1p2 = out.params['sigma_1p2'].value
        sig_alkyl = out.params['sigma_alkyl'].value
        sig_pipi = out.params['sigma_pipi'].value
              
        # ratios[f'lamella-pipi_sig_{round(chi_bin.left)}-{round(chi_bin.right)}'] = np.round(sig_lamella/sig_pipi, 2)
        # ratios[f'lamella-alkyl_sig_{round(chi_bin.left)}-{round(chi_bin.right)}'] = np.round(sig_lamella/sig_alkyl, 2)
        # if out.redchi<50 and np.abs(out.residual.mean())<1:
        #     good_ratios[f'{sn_id[binned_DA.sample_ID]}_{binned_DA.incident_angle}'] = ratios
        #     print(f'lamella-pipi sig ratio: {np.round(sig_lamella/sig_pipi, 2)}')
        #     print(f'lamella-alkyl sig ratio: {np.round(sig_lamella/sig_alkyl, 2)}')
        # else:
        #     print('bad fit, so dont compare these ratios')
        #     bad_ratios[f'{sn_id[binned_DA.sample_ID]}_{binned_DA.incident_angle}'] = ratios

        ax.plot(x, data, 'o')
        ax.plot(x, full_fit, '-', label='full_fit')
        ax.plot(x, lin_bkg, label=f'lin_bkg')
        ax.plot(x, exp_bkg, label=f'exp_bkg')
        ax.plot(x, pvoigt1, label=f'lamella: sig={sig_lamella}')
        ax.plot(x, pvoigt2, label=f'backbone: sig={sig_backbone}')
        ax.plot(x, pvoigt3, label=f'0p9: sig={sig_0p9}')
        ax.plot(x, pvoigt4, label=f'1p2: sig={sig_1p2}')
        ax.plot(x, pvoigt5, label=f'alkyl: sig={sig_alkyl}')
        ax.plot(x, pvoigt6, label=f'pipi: sig={sig_pipi}')
        ax.set_title(f'{sn[sel_DA.sample_ID]}_{chi_bin}')
        ax.legend(title=f'redchi={np.round(out.redchi, 1)}, abs_resid_mean={np.round(np.abs(out.residual).mean(), 1)}, resid_mean={np.round(out.residual.mean(), 1)}')

        plt.show()
        plt.close('all')

In [ ]:
# lamella_alkyl_ratios = []
# lamella_pipi_ratios = []
# for sample, ratios in good_ratios.items():
#     for ratio_chi, value in ratios.items():
#         ratio, sig, chi_bin = ratio_chi.split('_')
#         if ratio=='lamella-alkyl':
#             lamella_alkyl_ratios.append(value)
#         elif ratio=='lamella-pipi':
#             lamella_pipi_ratios.append(value)
            
# lamella_pipi_ratios

In [ ]:
# lamella_alkyl_ratios = []
# lamella_pipi_ratios = []
# for sample, ratios in bad_ratios.items():
#     for ratio_chi, value in ratios.items():
#         ratio, sig, chi_bin = ratio_chi.split('_')
#         if ratio=='lamella-alkyl':
#             lamella_alkyl_ratios.append(value)
#         elif ratio=='lamella-pipi':
#             lamella_pipi_ratios.append(value)
            
# lamella_pipi_ratios

#### Unused code for simultaneous fitting (slow, inefficient)

In [ ]:
# Define function that will loop over a dataset for fitting multiple lines simulataneously:
def linear_dataset(pars, x, i):
    slope_lin, intercept_lin = pars[f'slope_lin_{i}'].value, pars[f'intercept_lin_{i}'].value
    return linear(x, slope_lin, intercept_lin)

def exponential_dataset(pars, x, i):
    amp_exp, decay_exp = pars[f'amp_exp_{i}'].value, pars[f'decay_exp_{i}'].value
    return exponential(x, amp_exp, decay_exp)

def pvoigt_dataset(pars, x, i, peakname):
    amp, cen, sigma, fraction = pars[f'amp_{peakname}_{i}'].value, pars[f'cen_{peakname}_{i}'].value, pars[f'sigma_{peakname}_{i}'].value, pars[f'fraction_{peakname}_{i}'].value
    return pvoigt(x, amp, cen, sigma, fraction)
    
def total_func_dataset(pars, x, i):
    slope_lin, intercept_lin = pars[f'slope_lin_{i}'].value, pars[f'intercept_lin_{i}'].value
    amp_exp, decay_exp = pars[f'amp_exp_{i}'].value, pars[f'decay_exp_{i}'].value
    amp_lamella, cen_lamella, sigma_lamella, fraction_lamella = pars[f'amp_lamella_{i}'].value, pars[f'cen_lamella_{i}'].value, pars[f'sigma_lamella_{i}'].value, pars[f'fraction_lamella_{i}'].value
    amp_backbone, cen_backbone, sigma_backbone, fraction_backbone = pars[f'amp_backbone_{i}'].value, pars[f'cen_backbone_{i}'].value, pars[f'sigma_backbone_{i}'].value, pars[f'fraction_backbone_{i}'].value
    amp_0p9, cen_0p9, sigma_0p9, fraction_0p9 = pars[f'amp_0p9_{i}'].value, pars[f'cen_0p9_{i}'].value, pars[f'sigma_0p9_{i}'].value, pars[f'fraction_0p9_{i}'].value
    amp_1p2, cen_1p2, sigma_1p2, fraction_1p2 = pars[f'amp_1p2_{i}'].value, pars[f'cen_1p2_{i}'].value, pars[f'sigma_1p2_{i}'].value, pars[f'fraction_1p2_{i}'].value
    amp_alkyl, cen_alkyl, sigma_alkyl, fraction_alkyl = pars[f'amp_alkyl_{i}'].value, pars[f'cen_alkyl_{i}'].value, pars[f'sigma_alkyl_{i}'].value, pars[f'fraction_alkyl_{i}'].value
    amp_pipi, cen_pipi, sigma_pipi, fraction_pipi = pars[f'amp_pipi_{i}'].value, pars[f'cen_pipi_{i}'].value, pars[f'sigma_pipi_{i}'].value, pars[f'fraction_pipi_{i}'].value  
    
    return total_func(x, 
                      slope_lin, intercept_lin,
                      amp_exp, decay_exp,
                      amp_lamella, cen_lamella, sigma_lamella, fraction_lamella,
                      amp_backbone, cen_backbone, sigma_backbone, fraction_backbone,
                      amp_0p9, cen_0p9, sigma_0p9, fraction_0p9,
                      amp_1p2, cen_1p2, sigma_1p2, fraction_1p2,
                      amp_alkyl, cen_alkyl, sigma_alkyl, fraction_alkyl,
                      amp_pipi, cen_pipi, sigma_pipi, fraction_pipi)
    
    
def objective(pars, x, data):
    """Calculate total residual for fits of Gaussians to several data sets."""
    ndata, _ = data.shape
    resid = 0.0*data[:]

    # make residual per data set
    for i in range(ndata):
        resid[i, :] = data[i, :] - total_func_dataset(pars, x, i)

    # now flatten this to a 1D array, as minimize() needs
    return resid.flatten()

In [ ]:
# Select data linecut, create parameters, run fit:
# Goal of this cell to simultaneously fit parameters for all linecuts
%matplotlib inline
plt.close('all')

### Select data
# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
# chi_min, chi_width, chi_bins = [10, 6, 12]  
chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

q_min = 0.25  # for full qr cut
q_max = 1.75

# Select attribute
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
selected_attrs_dict = {'sample_ID': ['AL22', 'AL25', 'AL26'], 'incident_angle': ['th0.110']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

all_outs = {}
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi')
    
    x = binned_DA.qr.data
    data = binned_DA.data
    
    params = Parameters()   
    total_slices = 5
    for iy, y in enumerate(data[:total_slices]):
        ### Create initial parameter values & set constraints:
        params.add(f'slope_lin_{iy}',         value=0,     vary=False)
        params.add(f'intercept_lin_{iy}',     value=y.min(), min=y.min()/2, max=y.min()*2)
        params.add(f'amp_exp_{iy}',           value=0,     min=0,     max=1e2, vary=False)
        params.add(f'decay_exp_{iy}',         value=0,     min=0.01,  max=1e2, vary=False)
        params.add(f'amp_lamella_{iy}',       value=250,   min=0,     max=1e4)
        params.add(f'cen_lamella_{iy}',       value=0.31,  min=0.28,  max=0.34)
        params.add(f'sigma_lamella_{iy}',     value=0.05,  min=1e-5,  max=0.5)
        params.add(f'fraction_lamella_{iy}',  value=0.5,   min=0,     max=1)
        params.add(f'amp_backbone_{iy}',      value=0.005, min=0,     max=10)
        params.add(f'cen_backbone_{iy}',      value=0.66,  min=0.63,  max=0.7)
        params.add(f'sigma_backbone_{iy}',    value=1e-3,  min=1e-5,  max=0.4)
        params.add(f'fraction_backbone_{iy}', value=0.5,   min=0,     max=1)
        params.add(f'amp_0p9_{iy}',           value=1,     min=0,     max=10)
        params.add(f'cen_0p9_{iy}',           value=0.93,  min=0.85,  max=0.99)
        params.add(f'sigma_0p9_{iy}',         value=0.02,  min=1e-5,  max=0.4)
        params.add(f'fraction_0p9_{iy}',      value=0.5,   min=0,     max=1)
        params.add(f'amp_1p2_{iy}',           value=1,     min=0,     max=10)
        params.add(f'cen_1p2_{iy}',           value=1.2,   min=1.15,  max=1.25)
        params.add(f'sigma_1p2_{iy}',         value=0.5,   min=0,     max=1)
        params.add(f'fraction_1p2_{iy}',      value=0.5,   min=0,     max=1)
        params.add(f'amp_alkyl_{iy}',         value=50,    min=0,     max=400)
        params.add(f'cen_alkyl_{iy}',         value=1.4,   min=1.35,  max=1.45)
        params.add(f'sigma_alkyl_{iy}',       value=0.5,   min=0,     max=1)
        params.add(f'fraction_alkyl_{iy}',    value=0.5,   min=0,     max=1)
        params.add(f'amp_pipi_{iy}',          value=100,   min=0,     max=1000)
        params.add(f'cen_pipi_{iy}',          value=1.73,  min=1.65,  max=1.8)
        params.add(f'sigma_pipi_{iy}',        value=0.1,   min=0,     max=1)
        params.add(f'fraction_pipi_{iy}',     value=0.5,   min=0,     max=1)    
        
    out = minimize(objective, params, args=(x, data[:total_slices]))
    all_outs[f'{sn_id[binned_DA.sample_ID]}_{binned_DA.incident_angle}'] = out
    
    for iy, y in enumerate(data[:total_slices]):
        fig, ax = plt.subplots()
        full_fit = total_func_dataset(out.params, x, iy)
        lin_bkg = linear_dataset(out.params, x, iy)
        exp_bkg = exponential_dataset(out.params, x, iy)
        pvoigt1 = pvoigt_dataset(out.params, x, iy, 'lamella')
        pvoigt2 = pvoigt_dataset(out.params, x, iy, 'backbone')
        pvoigt3 = pvoigt_dataset(out.params, x, iy, '0p9')
        pvoigt4 = pvoigt_dataset(out.params, x, iy, '1p2')
        pvoigt5 = pvoigt_dataset(out.params, x, iy, 'alkyl')
        pvoigt6 = pvoigt_dataset(out.params, x, iy, 'pipi')

        ax.plot(x, y, 'o')
        ax.plot(x, full_fit, '-', label='full_fit')
        ax.plot(x, lin_bkg, label='lin_bkg')
        ax.plot(x, exp_bkg, label='exp_bkg')
        ax.plot(x, pvoigt1, label='lamella')
        ax.plot(x, pvoigt2, label='backbone')
        ax.plot(x, pvoigt3, label='0p9')
        ax.plot(x, pvoigt4, label='1p2')
        ax.plot(x, pvoigt5, label='alkyl')
        ax.plot(x, pvoigt6, label='pipi')
        ax.legend()

        plt.show()
        plt.close('all')

In [ ]:
for i in range(total_slices):
    intercept = out.params[f'intercept_lin_{i}']
    print(intercept)

In [ ]:
for i in range(total_slices):
    amp = out.params[f'amp_{peak}_{i}']
    cen = out.params[f'cen_{peak}_{i}']
    sigma = out.params[f'sigma_{peak}_{i}']
    print(amp)
    print(cen)
    print(sigma)


In [ ]:
peak = 'pipi'
for i in range(total_slices):
    amp = out.params[f'amp_{peak}_{i}']
    cen = out.params[f'cen_{peak}_{i}']
    sigma = out.params[f'sigma_{peak}_{i}']
    print(amp)
    print(cen)
    print(sigma)


In [ ]:
out.redchi

In [ ]:
out.params

In [ ]:
np.abs(out.residual).mean()

#### PM6 films old fitting functions below:

In [ ]:
def full_lmfit(sliced_DA):
    """
    Function currently utilizes global variables, define in notebook!
    
    """
    # point_x = float(sliced_DA.min())
    # point_y = float(sliced_DA.sel(qr=slice(point_x-0.01, point_x+0.01)).mean('qr'))
    
    # point_y = float(sliced_DA.sel(qr=slice(0.55,0.59)).mean('qr'))
    point_y = float(sliced_DA.min())
    point_y = point_y - point_y*(0.2)
    # point_y = point_y - 100

    x = sliced_DA.qr.data
    y = sliced_DA.data

    # Define all models to include in fitting
    bkg_mod = models.LinearModel(prefix='bkg_')
    pars = bkg_mod.make_params(intercept=point_y, slope=0)
    pars['bkg_intercept'].set(vary=False)
    pars['bkg_slope'].set(vary=False)
    
    exp_mod = models.ExponentialModel(prefix='exp_')
    pars += exp_mod.make_params(decay=0.1, amplitude=100)
    pars['exp_decay'].set(min=0.01)
    pars['exp_amplitude'].set(min=0.1)
    
    lamella_mod = models.PseudoVoigtModel(prefix='lamella_')  # lamella peak at 0.32
    pars += lamella_mod.make_params(center={'value':0.31, 'min':0.28, 'max':0.34}, 
                                    sigma ={'value':0.09/2, 'min':0.01/2, 'max':0.2/2})
    pars['lamella_amplitude'].set(min=30)
    
    backbone_mod = models.PseudoVoigtModel(prefix='backbone_')  # backbone peak at 0.66
    pars += backbone_mod.guess(y, x, center=0.66)
    pars['backbone_amplitude'].set(min=0, max=10)
    pars['backbone_center'].set(min=0.63, max=0.73)
    # pars['backbone_sigma'].set(max=0.1)

    pk0p9_mod = models.PseudoVoigtModel(prefix='pk0p9_')  # 300  peak around 0.9
    pars += pk0p9_mod.guess(y, x, center=0.93)
    pars['pk0p9_amplitude'].set(min=0)
    pars['pk0p9_center'].set(min=0.88, max=0.99)

    pk1p2_mod = models.PseudoVoigtModel(prefix='pk1p2_')  # 021? peak around 1.2
    pars += pk1p2_mod.guess(y, x, center=1.2)
    # pars += pk1p2_mod.make_params(center={'value':1.2, 'min':1.15, 'max':1.28}, 
    #                               sigma ={'value':0.8/2, 'min':0.01/2, 'max':3/2})
    pars['pk1p2_amplitude'].set(min=0)
    pars['pk1p2_center'].set(min=1.18, max=1.26)
    
    pkgroup_mod = models.PseudoVoigtModel(prefix='pkgroup_')  # broad collection of slip-stack peaks around 1.2-2
    pars += pkgroup_mod.guess(y, x, center=1.4)
    # pars += pkgroup_mod.make_params(center={'value':1.4, 'min':1.3, 'max':1.6}, 
    #                                 sigma ={'value':0.8/2, 'min':0.01/2, 'max':3/2})
    pars['pkgroup_center'].set(min=1.3, max=1.5)
    pars['pkgroup_amplitude'].set(min=10)
    pars['pkgroup_sigma'].set(min=0.1, max=0.4)

    pipi_mod = models.PseudoVoigtModel(prefix='pipi_')  # pi-pi peak! around 1.73
    pars += pipi_mod.guess(y, x, center=1.73)
    pars['pipi_amplitude'].set(min=0, max=500)
    pars['pipi_center'].set(min=1.7, max=1.76)
    pars['pipi_sigma'].set(max=0.3)
    
    pk1p86_mod = models.PseudoVoigtModel(prefix='pk1p86_')  # highq peak
    pars += pk1p86_mod.guess(y, x, center=1.86)
    pars['pk1p86_amplitude'].set(min=0, max=80)
    pars['pk1p86_center'].set(min=1.84, max=1.9)

    # Combine into full model
    mod = bkg_mod + exp_mod + lamella_mod + backbone_mod + pk0p9_mod + pk1p2_mod + pkgroup_mod + pipi_mod + pk1p86_mod

    # Run fit and store all info in a ModelResult object
    out = mod.fit(y, pars, x=x)
    return out

def full_fit(sliced_DA, show_plot=True):   
    # Run lmfit
    out = full_lmfit(sliced_DA)
    # FWHM = np.round(float(out.params['pk1_fwhm']), 2)
    # Lc = np.round((2*np.pi*0.9)/float(out.params['pk1_fwhm']), 2)
    # center = np.round(float(out.params['pk1_center']), 2)
    # dspacing = np.round((2*np.pi)/float(out.params['pk1_center']), 2)
    
    # amplitude = np.round(float(out.params['pk1_amplitude']), 3)
    
    rsquared = np.round(out.rsquared, 3)
    redchi = np.round(out.redchi, 4)

    AA1 = '$\AA^{-1}$'
    
    # Plot
    q = sliced_DA.qr.data
    I = sliced_DA.data
    
    fig, ax = plt.subplots()
    fig.set(size_inches=(6.5,3.5), dpi=120, tight_layout=True)
    fig.suptitle(
        f'Full fit: {sn[sliced_DA.sample_ID]} {sliced_DA.sample_pos}, $\\alpha$={float(sliced_DA.incident_angle[2:])}°; {sliced_DA.chi_bin}° chi',
        fontsize=12, y=0.93, x=0.53)
       
    ax.plot(q, I, label='data', linewidth=2.5)
    ax.plot(q, out.best_fit, '--', label='full_fit')
    for key in out.eval_components():
        ax.plot(q, out.eval_components()[key], label=f'{key}')
    ax.set(xlabel=f'Q [{AA1}]', ylabel='Intensity [arb. units]', yscale='linear')
    # ax.set_title(
    #     f'Center = {center}, FWHM = {FWHM} {AA1} (d-spacing = {dspacing}, $L_c$ = {Lc} $\AA$)')
    ax.legend(title=f'$R^2$ = {rsquared}, $\chi_r$ = {redchi}', loc='upper left', bbox_to_anchor=(1,1.05))
    ax.grid(visible=True, axis='x')
        
    
    # FWHM = np.round(float(out.params['pk1_fwhm']), 2)
    # Lc = np.round((2*np.pi*0.9)/FWHM, 2)  # calculate scherrer coherence length

    if show_plot:
        plt.show()
    
    return out, fig, ax

In [ ]:
%matplotlib inline
plt.close('all')

# Peak fitting for each chi bin

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

q_min = 0.2
q_max = 1.86

# Select attributes
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26',
                                     'AL31', 'AL41', 'AL28', 'AL38'], 
                       'incident_angle': ['th0.080']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

# For each selected DA, fit the π-π peak
all_outs = {}
all_params = {}
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi').compute()
    outs = {}
    params = {}
    for i, chi_bin in tqdm(enumerate(binned_DA.chi_bins.data[::1]), desc='chi_bins', total=len(binned_DA.chi_bins.data)):
        key = f'{str(round(chi_bin.left))}-{str(round(chi_bin.right))}'
        sel_DA = binned_DA.sel(chi_bins=chi_bin)
        sel_DA.attrs['chi_bin'] = chi_bin
        outs[key], fig, ax = full_fit(sel_DA)
        params[key] = outs[key].params.valuesdict()

        # Save paths
        outPath.joinpath('full_linefits_v1').mkdir(exist_ok=True)
        savePath = outPath.joinpath('full_linefits_v1', f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
        savePath.mkdir(exist_ok=True)
        sampleFitPath = savePath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}')
        sampleFitPath.mkdir(exist_ok=True)
                                          
        # Save plot 
        fig.savefig(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_chi{key}_pi-pi_fit.png'), dpi=150)

        plt.show()
        plt.close('all')
                         
    # Save fit params:
    json_data = json.dumps(params)
    with open(str(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_params.json')), 'w') as f:
        f.write(json_data)   
                                          
    # Save fit results
    for chi_bin, out in outs.items():
        with sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_{chi_bin}_fit_result.txt').open(mode='w') as f:
            f.write(out.fit_report())
        
    # Append to full in-memory dictionaries
    all_outs[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}'] = outs
    all_params[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}'] = params

# # Save model_params.json in save folder for all samples
# json_data = json.dumps(all_params)
# with open(str(savePath.joinpath('model_params.json')), 'w') as f:
#     f.write(json_data)

# # Create full fit reports folder and save entire report .txt files inside:
# savePath.joinpath('fit_reports').mkdir(exist_ok=True)
# for film, outs in all_outs.items():
#     for chi_bin, out in outs.items():
#         with savePath.joinpath('fit_reports', f'{film}_{chi_bin}_fit_result.txt').open(mode='w') as f:
#             f.write(out.fit_report())

#### PM6 (GPT) updated fitting functions below:

In [ ]:
import lmfit

In [ ]:
models.PseudoVoigtModel??

In [ ]:
def build_full_model(sliced_DA, prev_params=None):
    """
    Build lmfit model with initial parameter guesses, optionally using previous fit results.
    """
    x = sliced_DA.qr.data
    y = sliced_DA.data
    
    # Define a dictionary for peak models
    peak_models = {
        'bkg': models.LinearModel(prefix='bkg_'),
        'exp': models.ExponentialModel(prefix='exp_'),
        'lamella': models.PseudoVoigtModel(prefix='lamella_'),
        'backbone': models.PseudoVoigtModel(prefix='backbone_'),
        'pk0p9': models.PseudoVoigtModel(prefix='pk0p9_'),
        'pk1p2': models.PseudoVoigtModel(prefix='pk1p2_'),
        'pkgroup': models.PseudoVoigtModel(prefix='pkgroup_'),
        'pipi': models.PseudoVoigtModel(prefix='pipi_'),
        # 'pk1p86': models.PseudoVoigtModel(prefix='pk1p86_')
    }

    # Set up initial parameter guesses (use previous fit parameters if available)
    pars = lmfit.Parameters()
    
    # Add background model
    point_y = float(sliced_DA.min()) * 0.02  # Modify the initial y value
    pars += peak_models['bkg'].make_params(intercept=point_y, slope=0)
    pars['bkg_intercept'].set(vary=False)
    pars['bkg_slope'].set(vary=False)

    # Add exponential model
    pars += peak_models['exp'].make_params(decay=0.1, amplitude=100)
    pars['exp_decay'].set(min=0.01)
    pars['exp_amplitude'].set(min=0.1)

    # Add peaks, updating with previous parameters if available
    peak_definitions = [
        {'name': 'lamella', 
         'center': (0.31, 0.28, 0.34), 
         'amplitude_min': 30},
        {'name': 'backbone', 
         'center': (0.66, 0.63, 0.73), 
         'amplitude_min': 0, 
         'amplitude_max': 10},
        {'name': 'pk0p9', 
         'center': (0.93, 0.88, 0.99), 
         'amplitude_min': 0},
        {'name': 'pk1p2', 
         'center': (1.2, 1.18, 1.22), 
         'amplitude_min': 0},
        {'name': 'pkgroup', 
         'center': (1.4, 1.35, 1.45), 
         'amplitude_min': 10},
        {'name': 'pipi', 
         'center': (1.73, 1.7, 1.76), 
         'amplitude_min': 0, 
         'amplitude_max': 500},
        # {'name': 'pk1p86', 'center': (1.86, 1.84, 1.9), 'amplitude_min': 0, 'amplitude_max': 80}
    ]

    for peak in peak_definitions:
        model_name = peak['name']
        model = peak_models[model_name]
        center, center_min, center_max = peak['center']
        pars += model.guess(y, x, center=center)
        pars[f'{model_name}_center'].set(min=center_min, max=center_max)
        pars[f'{model_name}_amplitude'].set(min=peak.get('amplitude_min', 0), max=peak.get('amplitude_max', None))
        if prev_params:  # Use previous fit results to update initial guesses
            pars[f'{model_name}_center'].set(value=prev_params.get(f'{model_name}_center', center))
            pars[f'{model_name}_amplitude'].set(value=prev_params.get(f'{model_name}_amplitude', pars[f'{model_name}_amplitude'].value))

    # Combine all models
    mod = sum([m for name, m in peak_models.items() if name != 'bkg'], peak_models['bkg'])  # Sum all models starting with the background
    return mod, pars

def plot_fit_results(out, sliced_DA):
    """
    Function to plot fit results and original data.
    """
    q = sliced_DA.qr.data
    I = sliced_DA.data
    
    fig, ax = plt.subplots()
    fig.set(size_inches=(6.5, 3.5), dpi=120, tight_layout=True)
    fig.suptitle(f'Fit: {sn[sliced_DA.sample_ID]} {sliced_DA.sample_pos}, '
                 f'alpha={float(sliced_DA.incident_angle[2:])}°, {sliced_DA.chi_bin}° chi', fontsize=12, y=0.93, x=0.53)
    
    ax.plot(q, I, label='Data', linewidth=2.5)
    ax.plot(q, out.best_fit, '--', label='Fit')
    
    # Plot each component
    for key, comp in out.eval_components().items():
        ax.plot(q, comp, label=f'{key}')
    
    ax.set(xlabel='Q [$\AA^{-1}$]', ylabel='Intensity [arb. units]', yscale='linear')
    ax.legend(title=f'$R^2$ = {out.rsquared:.3f}, $\chi_r$ = {out.redchi:.4f}', loc='upper left', bbox_to_anchor=(1, 1.05))
    ax.grid(True, axis='x')
    
    plt.show()

def fit_peaks_across_chi_bins(binned_DA, prev_fit_results=None):
    """
    Fit peaks across chi bins, updating initial guesses with the previous fit's results.
    """
    outs = {}
    params = {}
    prev_params = prev_fit_results or {}
    
    for i, chi_bin in tqdm(enumerate(binned_DA.chi_bins.data), desc='chi_bins', total=len(binned_DA.chi_bins.data)):
        key = f'{round(chi_bin.left)}-{round(chi_bin.right)}'
        sel_DA = binned_DA.sel(chi_bins=chi_bin)
        sel_DA.attrs['chi_bin'] = chi_bin
        
        # Build model and run fit using previous parameters
        model, pars = build_full_model(sel_DA, prev_params=prev_params)
        out = model.fit(sel_DA.data, pars, x=sel_DA.qr.data)
        outs[key] = out
        
        # Store parameters for updating the next bin's initial guesses
        prev_params = out.params.valuesdict()
        params[key] = prev_params
        
        # Optionally plot
        plot_fit_results(out, sel_DA)

    return outs, params

In [ ]:
%matplotlib inline
plt.close('all')

# Full fit for each chi bin

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
# chi_min, chi_width, chi_bins = [10, 8, 9]  
chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

q_min = 0.2
q_max = 1.82

# Select attributes
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
selected_attrs_dict = {'sample_ID': ['AL22'], 
                       'incident_angle': ['th0.110']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

# For each selected DA, fit the π-π peak
all_outs = {}
all_params = {}
prev_fit_results = None
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi').compute()
    
    outs, params = fit_peaks_across_chi_bins(binned_DA, prev_fit_results=prev_fit_results)
    prev_fit_results = params
    
        
    # Append to in-memory dictionaries
    all_outs[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}'] = outs
    all_params[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}'] = params

#         # Save paths
#         outPath.joinpath('full_linefits_v1').mkdir(exist_ok=True)
#         savePath = outPath.joinpath('full_linefits_v1', f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
#         savePath.mkdir(exist_ok=True)
#         sampleFitPath = savePath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}')
#         sampleFitPath.mkdir(exist_ok=True)
                                          
#         # Save plot 
#         fig.savefig(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_chi{key}_pi-pi_fit.png'), dpi=150)

#         plt.show()
#         plt.close('all')
                         
#     # Save fit params:
#     json_data = json.dumps(params)
#     with open(str(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_params.json')), 'w') as f:
#         f.write(json_data)   
                                          
#     # Save fit results
#     for chi_bin, out in outs.items():
#         with sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_{chi_bin}_fit_result.txt').open(mode='w') as f:
#             f.write(out.fit_report())


In [ ]:
def build_pipi_model(sliced_DA, prev_params=None, pipi_qlims=None):
    """
    Build lmfit model with initial parameter guesses, optionally using previous fit results.
    """
    x = sliced_DA.sel(qr=slice(pipi_qlims[0], pipi_qlims[1])).qr.data
    y = sliced_DA.sel(qr=slice(pipi_qlims[0], pipi_qlims[1])).data
    
    # Define a dictionary for peak models
    peak_models = {
        'bkg': models.LinearModel(prefix='bkg_'),
        'pkgroup': models.PseudoVoigtModel(prefix='pkgroup_'),
        'pipi': models.PseudoVoigtModel(prefix='pipi_'),
        'pk1p86': models.PseudoVoigtModel(prefix='pk1p86_')
    }

    # Set up initial parameter guesses (use previous fit parameters if available)
    pars = lmfit.Parameters()
    
    # Add background model
    point_y = float(sliced_DA.min()) * 0.8  # Modify the initial y value
    pars += peak_models['bkg'].make_params(intercept=point_y, slope=0)
    pars['bkg_intercept'].set(vary=False)
    pars['bkg_slope'].set(vary=False)

    # Add peaks, updating with previous parameters if available
    peak_definitions = [
        # {'name': 'pkgroup', 'center': (1.4, 1.35, 1.45), 'amplitude_min': 50, 'sigma_min': 0.2, 'sigma_max': 0.5},
        {'name': 'pipi', 'center': (1.72, 1.705, 1.735), 'amplitude_min': 0, 'sigma_min': 0.1, 'sigma_max': 0.3},
        # {'name': 'pk1p86', 'center': (1.86, 1.84, 1.9), 'amplitude_min': 0, 'amplitude_max': 80, 'sigma_min': 0.2, 'sigma_max': 0.5}
        {'name': 'pkgroup', 'center': (1.4, 1.35, 1.45), 'amplitude_min': 0},
        # {'name': 'pipi', 'center': (1.72, 1.69, 1.75), 'amplitude_min': 0},
        {'name': 'pk1p86', 'center': (1.86, 1.84, 1.9), 'amplitude_min': 0}
    ]

    for peak in peak_definitions:
        model_name = peak['name']
        model = peak_models[model_name]
        center, center_min, center_max = peak['center']
        pars += model.guess(y, x, center=center)
        pars[f'{model_name}_center'].set(min=center_min, max=center_max)
        pars[f'{model_name}_amplitude'].set(min=peak.get('amplitude_min', 0), max=peak.get('amplitude_max', None))
        # pars[f'{model_name}_sigma'].set(min=peak.get('sigma_min', 0), max=peak.get('sigma_max', None))
        if prev_params:  # Use previous fit results to update initial guesses
            pars[f'{model_name}_center'].set(value=prev_params.get(f'{model_name}_center', center))
            pars[f'{model_name}_amplitude'].set(value=prev_params.get(f'{model_name}_amplitude', pars[f'{model_name}_amplitude'].value))

    # Combine all models
    mod = sum([m for name, m in peak_models.items() if name != 'bkg'], peak_models['bkg'])  # Sum all models starting with the background
    return mod, pars

def plot_fit_results(out, sliced_DA):
    """
    Function to plot fit results and original data.
    """
    q = sliced_DA.qr.data
    I = sliced_DA.data
    
    fig, ax = plt.subplots()
    fig.set(size_inches=(6.5, 3.5), dpi=120, tight_layout=True)
    fig.suptitle(f'Fit: {sn[sliced_DA.sample_ID]} {sliced_DA.sample_pos}, '
                 f'alpha={float(sliced_DA.incident_angle[2:])}°, {sliced_DA.chi_bin}° chi', fontsize=12, y=0.93, x=0.53)
    
    ax.plot(q, I, label='Data', linewidth=2.5)
    ax.plot(q, out.best_fit, '--', label='Fit')
    
    # Plot each component
    for key, comp in out.eval_components().items():
        ax.plot(q, comp, label=f'{key}')
    
    ax.set(xlabel='Q [$\AA^{-1}$]', ylabel='Intensity [arb. units]', yscale='linear')
    ax.legend(title=f'$R^2$ = {out.rsquared:.3f}, $\chi_r$ = {out.redchi:.4f}', loc='upper left', bbox_to_anchor=(1, 1.05))
    ax.grid(True, axis='x')
    
    plt.show()

def fit_pipi_across_chi_bins(binned_DA, prev_fit_results=None, pipi_qlims=None):
    """
    Fit peaks across chi bins, updating initial guesses with the previous fit's results.
    """
    outs = {}
    params = {}
    prev_params = prev_fit_results or {}
    
    for i, chi_bin in tqdm(enumerate(binned_DA.chi_bins.data), desc='chi_bins', total=len(binned_DA.chi_bins.data)):
        key = f'{round(chi_bin.left)}-{round(chi_bin.right)}'
        sel_DA = binned_DA.sel(chi_bins=chi_bin)
        sel_DA.attrs['chi_bin'] = chi_bin
        
        # Build model and run fit using previous parameters
        model, pars = build_pipi_model(sel_DA, prev_params=prev_params, pipi_qlims=pipi_qlims)
        out = model.fit(sel_DA.sel(qr=slice(pipi_qlims[0], pipi_qlims[1])).data, pars, x=sel_DA.sel(qr=slice(pipi_qlims[0], pipi_qlims[1])).qr.data)
        outs[key] = out
        
        # Store parameters for updating the next bin's initial guesses
        prev_params = out.params.valuesdict()
        params[key] = prev_params
        
        # Optionally plot
        plot_fit_results(out, sel_DA.sel(qr=slice(pipi_qlims[0], pipi_qlims[1])))

    return outs, params

In [ ]:
%matplotlib inline
plt.close('all')

# pipi peak fitting for each chi bin

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
chi_min, chi_width, chi_bins = [10, 8, 9]  
# chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

q_min = 0.2
q_max = 1.86

# Select attributes
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
selected_attrs_dict = {'sample_ID': ['AL22'], 
                       'incident_angle': ['th0.110']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

# For each selected DA, fit the π-π peak
all_outs = {}
all_params = {}
prev_fit_results = None
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi').compute()
    
    outs, params = fit_pipi_across_chi_bins(binned_DA, prev_fit_results=prev_fit_results, pipi_qlims=(1.4, 1.86))
    prev_fit_results = params
    
        
    # Append to in-memory dictionaries
    all_outs[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}'] = outs
    all_params[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}'] = params

#         # Save paths
#         outPath.joinpath('full_linefits_v1').mkdir(exist_ok=True)
#         savePath = outPath.joinpath('full_linefits_v1', f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
#         savePath.mkdir(exist_ok=True)
#         sampleFitPath = savePath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}')
#         sampleFitPath.mkdir(exist_ok=True)
                                          
#         # Save plot 
#         fig.savefig(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_chi{key}_pi-pi_fit.png'), dpi=150)

#         plt.show()
#         plt.close('all')
                         
#     # Save fit params:
#     json_data = json.dumps(params)
#     with open(str(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_params.json')), 'w') as f:
#         f.write(json_data)   
                                          
#     # Save fit results
#     for chi_bin, out in outs.items():
#         with sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_{chi_bin}_fit_result.txt').open(mode='w') as f:
#             f.write(out.fit_report())


In [ ]:
sigmas = np.array([])
centers = np.array([])
amplitudes = np.array([])

for key in params.keys():
    # print(params[key]['pkgroup_sigma'])
    sigmas = np.append(sigmas, params[key]['pkgroup_sigma'])
    centers = np.append(centers, params[key]['pkgroup_center'])
    amplitudes = np.append(amplitudes, params[key]['pkgroup_amplitude'])
    
print(sigmas.mean())
print(centers.mean())
print(amplitudes.mean())

In [ ]:
peak = 'pipi'
sigmas = np.array([])
centers = np.array([])
amplitudes = np.array([])

for key in params.keys():
    sigmas = np.append(sigmas, params[key][f'{peak}_sigma'])
    centers = np.append(centers, params[key][f'{peak}_center'])
    amplitudes = np.append(amplitudes, params[key][f'{peak}_amplitude'])
    
print(sigmas.mean())
print(centers.mean())
print(amplitudes.mean())

In [ ]:
peak = 'pk1p86'
sigmas = np.array([])
centers = np.array([])
amplitudes = np.array([])

for key in params.keys():
    sigmas = np.append(sigmas, params[key][f'{peak}_sigma'])
    centers = np.append(centers, params[key][f'{peak}_center'])
    amplitudes = np.append(amplitudes, params[key][f'{peak}_amplitude'])
    
print(sigmas.mean())
print(centers.mean())
print(amplitudes.mean())

In [ ]:
sigmas = np.array([])
centers = np.array([])
amplitudes = np.array([])

for key in params.keys():
    # print(params[key]['pkgroup_sigma'])
    sigmas = np.append(sigmas, params[key]['pkgroup_sigma'])
    centers = np.append(centers, params[key]['pkgroup_center'])
    amplitudes = np.append(amplitudes, params[key]['pkgroup_amplitude'])
    
print(sigmas.mean())
print(centers.mean())
print(amplitudes.mean())

In [ ]:
sigmas.mean()

In [ ]:
%matplotlib inline
plt.close('all')

# Pipi peak fitting for each chi bin

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

q_min = 0.2
q_max = 1.86

# Select attributes
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
selected_attrs_dict = {'sample_ID': ['AL22'], 
                       'incident_angle': ['th0.110']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

# For each selected DA, fit the π-π peak
all_outs = {}
all_params = {}
prev_fit_results = None
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi').compute()
    
    outs, params = fit_pipi_across_chi_bins(binned_DA, prev_fit_results=prev_fit_results, pipi_qlims=)
    prev_fit_results = params
    
        
    # Append to in-memory dictionaries
    all_outs[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}'] = outs
    all_params[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}'] = params

#         # Save paths
#         outPath.joinpath('full_linefits_v1').mkdir(exist_ok=True)
#         savePath = outPath.joinpath('full_linefits_v1', f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
#         savePath.mkdir(exist_ok=True)
#         sampleFitPath = savePath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}')
#         sampleFitPath.mkdir(exist_ok=True)
                                          
#         # Save plot 
#         fig.savefig(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_chi{key}_pi-pi_fit.png'), dpi=150)

#         plt.show()
#         plt.close('all')
                         
#     # Save fit params:
#     json_data = json.dumps(params)
#     with open(str(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_params.json')), 'w') as f:
#         f.write(json_data)   
                                          
#     # Save fit results
#     for chi_bin, out in outs.items():
#         with sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_{chi_bin}_fit_result.txt').open(mode='w') as f:
#             f.write(out.fit_report())


#### New fiting code (inspired from Tom's notebook)

In [ ]:
from lmfit import Model, Parameters
from lmfit.model import save_modelresult, load_modelresult

In [ ]:
# Define generic functions for model:
def linear(x, m, b):
    """
    1-degree polynomial (straight line)
    
    e.g.: f(x) = mx + b
    
    Inputs:
        x: array-like, independent variable for function
        m: slope 
        b: y intercept value
    """
    return m*x + b 

def exp(x, A, d):
    """
    Exponential decay function: f(x) = A * exp(-d * x)
    
    Inputs:
        x: array-like, independent variable for function
        A: initial amplitude (value at x=0)
        d: decay value (rate value, larger values lead to faster decay)
    """
    return A * np.exp(-d * x)

def lorentz_peak(x, A, sigma, x0):
    """
    Lorentzian peak function

    Parameters:
    x : array_like, The independent variable where the Lorentz peak is evaluated.
    A : float, Amplitude of the Lorentzian peak (integrated scattering intensity).
    sigma : float, Half-width at half-maximum (HWHM) of the Lorentzian peak.
    x0 : float, Position of the center of the Lorentzian peak.

    Returns:
    array_like, Values of the Lorentzian function at each x.
    """
    return (A / np.pi) * (sigma / ((x - x0)**2 + sigma**2))

def gaussian_peak(x, A, sigma, x0):
    """
    Gaussian peak function
    
    Parameters:
    """
    return (A / (sigma * np.sqrt(2 * np.pi))) * np.exp((x - x0)**2 / (2 * sigma**2))

def pseudovoigt(x, A, sigma, x0, fraction):
    """
    Pseudo-Voigt peak function
    """
    return (fraction * gaussian_peak(x, A, sigma, x0)) + ((1 - fraction) * lorentz_peak(x, A, sigma, x0))

In [ ]:
# Define full PM6 GIWAXS model function:
def PM6_model_custom(x, m, b, exp_A, d, 
              amp1, sigma1, center1, fraction1,
              amp2, sigma2, center2, fraction2,
              amp3, sigma3, center3, fraction3,
              amp4, sigma4, center4, fraction4,
              amp5, sigma5, center5, fraction5,
              amp6, sigma6, center6, fraction6):
    """
    Combines: 1 linear background, 1 exponential decay background, and 6 Pseudo-Voigt peaks
    """
    
    lin_bkg = linear(x, m, b)
    exp_bkg = exp(x, exp_A, d)
    
    lamella =  pseudovoigt(x, amp1, sigma1, center1, fraction1)
    backbone = pseudovoigt(x, amp2, sigma2, center2, fraction2)
    pk0p9 =    pseudovoigt(x, amp3, sigma3, center3, fraction3)
    pk1p2 =    pseudovoigt(x, amp4, sigma4, center4, fraction4)
    pkalkyl =  pseudovoigt(x, amp5, sigma5, center5, fraction5)
    pipi =     pseudovoigt(x, amp6, sigma6, center6, fraction6)
    
    return lin_bkg + exp_bkg + lamella + backbone + pk0p9 + pk1p2 + pkalkyl + pipi

def PM6_model_lmfit(x, m, b, exp_A, d, 
              amp1, sigma1, center1, fraction1,
              amp2, sigma2, center2, fraction2,
              amp3, sigma3, center3, fraction3,
              amp4, sigma4, center4, fraction4,
              amp5, sigma5, center5, fraction5,
              amp6, sigma6, center6, fraction6):
    """
    Combines: 1 linear background, 1 exponential decay background, and 6 Pseudo-Voigt peaks
    """
    
    # Define all models to include in fitting
    bkg_mod = models.LinearModel(prefix='bkg_')
    pars = bkg_mod.make_params(intercept=b, slope=m)
    pars['bkg_intercept'].set(vary=False)
    pars['bkg_slope'].set(vary=False)
    
    exp_mod = models.ExponentialModel(prefix='exp_')
    pars += exp_mod.make_params(decay=d, amplitude=exp_A)
    pars['exp_decay'].set(min=0.01)
    pars['exp_amplitude'].set(min=0.1)
    
    lamella_mod = models.PseudoVoigtModel(prefix='lamella_')  # lamella peak at 0.32
    pars += lamella_mod.make_params(center={'value':0.31, 'min':0.28, 'max':0.34}, 
                                    sigma ={'value':0.09/2, 'min':0.01/2, 'max':0.2/2})
    pars['lamella_amplitude'].set(min=30)
    
    backbone_mod = models.PseudoVoigtModel(prefix='backbone_')  # backbone peak at 0.66
    pars += backbone_mod.guess(y, x, center=0.66)
    pars['backbone_amplitude'].set(min=0, max=10)
    pars['backbone_center'].set(min=0.63, max=0.73)
    # pars['backbone_sigma'].set(max=0.1)

    pk0p9_mod = models.PseudoVoigtModel(prefix='pk0p9_')  # 300  peak around 0.9
    pars += pk0p9_mod.guess(y, x, center=0.93)
    pars['pk0p9_amplitude'].set(min=0)
    pars['pk0p9_center'].set(min=0.88, max=0.99)

    pk1p2_mod = models.PseudoVoigtModel(prefix='pk1p2_')  # 021? peak around 1.2
    pars += pk1p2_mod.guess(y, x, center=1.2)
    # pars += pk1p2_mod.make_params(center={'value':1.2, 'min':1.15, 'max':1.28}, 
    #                               sigma ={'value':0.8/2, 'min':0.01/2, 'max':3/2})
    pars['pk1p2_amplitude'].set(min=0)
    pars['pk1p2_center'].set(min=1.18, max=1.26)
    
    pkgroup_mod = models.PseudoVoigtModel(prefix='pkgroup_')  # broad collection of slip-stack peaks around 1.2-2
    pars += pkgroup_mod.guess(y, x, center=1.4)
    # pars += pkgroup_mod.make_params(center={'value':1.4, 'min':1.3, 'max':1.6}, 
    #                                 sigma ={'value':0.8/2, 'min':0.01/2, 'max':3/2})
    pars['pkgroup_center'].set(min=1.3, max=1.5)
    pars['pkgroup_amplitude'].set(min=10)
    pars['pkgroup_sigma'].set(min=0.1, max=0.4)

    pipi_mod = models.PseudoVoigtModel(prefix='pipi_')  # pi-pi peak! around 1.73
    pars += pipi_mod.guess(y, x, center=1.73)
    pars['pipi_amplitude'].set(min=0, max=500)
    pars['pipi_center'].set(min=1.7, max=1.76)
    pars['pipi_sigma'].set(max=0.3)
    
    pk1p86_mod = models.PseudoVoigtModel(prefix='pk1p86_')  # highq peak
    pars += pk1p86_mod.guess(y, x, center=1.86)
    pars['pk1p86_amplitude'].set(min=0, max=80)
    pars['pk1p86_center'].set(min=1.84, max=1.9)

    # Combine into full model
    mod = bkg_mod + exp_mod + lamella_mod + backbone_mod + pk0p9_mod + pk1p2_mod + pkgroup_mod + pipi_mod + pk1p86_mod
    
    return lin_bkg + exp_bkg + lamella + backbone + pk0p9 + pk1p2 + pkalkyl + pipi

In [ ]:
# Initalize lmfit model and set default parameter values
model = Model(PM6_model)

# Create a Parameters object and set initial values and constraints
# variables must match arguments for fitting function
# variables are defined as list ['variable_name', start_value, lower_bound, upper_bound, Allow to vary?(True/False)]
m = ['m', 0, 0, np.inf, False]
b = ['b', 0, 0, np.inf, False]  # will likely need to revise later to be based on a diffuse value in the data
exp_A = ['exp_A', 2000, 0, 5000, True]
d = ['d', 1, 0.01, 100, True]

amp1 = ['amp1', 1000, 50, 1e3, True]  # lamella
amp2 = ['amp2', 50, 0, 1e3, True]  # backbone
amp3 = ['amp3', 10, 0, 1e3, True]  # pk0p9
amp4 = ['amp4', 5, 0, 1e3, True]  # pk1p2
amp5 = ['amp5', 40, 0, 1e3, True]  # pkalkyl
amp6 = ['amp6', 30, 0, 1e3, True]  # pipi

sigma1 = ['sigma1', 0.05, 0.01, 0.5, True]  # lamella
sigma2 = ['sigma2', 0.01, 0.005, 0.5, True]  # backbone
sigma3 = ['sigma3', 0.1, 0.01, 0.5, True]  # pk0p9
sigma4 = ['sigma4', 0.1, 0.01, 0.5, True]  # pk1p2
sigma5 = ['sigma5', 0.3, 0.01, 0.5, True]  # pkalkyl
sigma6 = ['sigma6', 0.1, 0.01, 0.5, True]  # pipi

center1 = ['center1', 0.31, 0.26, 0.35, True]  # lamella
center2 = ['center2', 0.66, 0.63, 0.73, True]  # backbone
center3 = ['center3', 0.93, 0.88, 0.99, True]  # pk0p9
center4 = ['center4', 1.20, 1.18, 1.22, True]  # pk1p2
center5 = ['center5', 1.40, 1.35, 1.45, True]  # pkalkyl
center6 = ['center6', 1.73, 1.70, 1.76, True]  # pipi

fraction1 = ['fraction1', 0.5, 0, 1, True]  # lamella
fraction2 = ['fraction2', 0.5, 0, 1, True]  # backbone
fraction3 = ['fraction3', 0.5, 0, 1, True]  # pk0p9
fraction4 = ['fraction4', 0.5, 0, 1, True]  # pk1p2
fraction5 = ['fraction5', 0.5, 0, 1, True]  # pkalkyl
fraction6 = ['fraction6', 0.5, 0, 1, True]  # pipi

# List all above variables (maybe this should just be a dictionary with the name of each being the key?)
variable_list = [m, b, exp_A, d, 
                 amp1, amp2, amp3, amp4, amp5, amp6,
                 sigma1, sigma2, sigma3, sigma4, sigma5, sigma6,
                 center1, center2, center3, center4, center5, center6,
                 fraction1, fraction2, fraction3, fraction4, fraction5, fraction6]

# Create lmfit parameters:
params = Parameters()
for var in variable_list:
    params.add(var[0], value=var[1], min=var[2], max=var[3], vary=var[4])
    
# Define weighting if desired (or set to None):
weights = None

params

In [ ]:
# Apply model + params to fit GIWAXS data IvsQ line cuts (binned/meaned along chi):
%matplotlib inline
plt.close('all')

# Full fit for each chi bin

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
# chi_min, chi_width, chi_bins = [10, 8, 9]  
chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

# Set q limits
q_min = 0.2
q_max = 1.82

# Select attributes
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
selected_attrs_dict = {'sample_ID': ['AL22'], 
                       'incident_angle': ['th0.110']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

# For each selected DA, fit the π-π peak
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi').compute()
    
    for i, chi_bin in tqdm(enumerate(binned_DA.chi_bins.data), desc='chi_bins', total=len(binned_DA.chi_bins.data)):
        key = f'{str(round(chi_bin.left))}-{str(round(chi_bin.right))}'
        sel_DA = binned_DA.sel(chi_bins=chi_bin)
        sel_DA.attrs['chi_bin'] = chi_bin
        
        x = sel_DA.qr.data
        y = sel_DA.data
        # result = model.fit(data=y, params=params, weights=weights, x=x, method='leastsq')
        result = model.fit(y, params=params, x=x, nan_policy='omit')

In [ ]:
result

In [ ]:
%matplotlib inline
plt.close('all')

# Peak fitting for each chi bin

# Set chi bounds & bins, q bounds, etc.
# chi_min, chi_width, chi_bins = [10, 24, 3] 
# chi_min, chi_width, chi_bins = [10, 12, 6]  
chi_min, chi_width, chi_bins = [10, 4, 18]  
chi_max = chi_min + (chi_width * chi_bins)
colors = plt.cm.viridis_r(np.linspace(0.15,1,chi_bins))

q_min = 0.2
q_max = 1.86

# Select attributes
# selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26', 
#                                      'AL31', 'AL41', 'AL28', 'AL38']}
selected_attrs_dict = {'sample_ID': ['AL22', 'AL36', 'AL25', 'AL26',
                                     'AL31', 'AL41', 'AL28', 'AL38'], 
                       'incident_angle': ['th0.080']}
# selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

# For each selected DA, fit the π-π peak
all_outs = {}
all_params = {}
for DA in tqdm(selected_DAs):
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min,q_max))
    binned_DA = sliced_DA.groupby_bins('chi', chi_bins).sum('chi').compute()
    outs = {}
    params = {}
    for i, chi_bin in tqdm(enumerate(binned_DA.chi_bins.data[::1]), desc='chi_bins', total=len(binned_DA.chi_bins.data)):
        key = f'{str(round(chi_bin.left))}-{str(round(chi_bin.right))}'
        sel_DA = binned_DA.sel(chi_bins=chi_bin)
        sel_DA.attrs['chi_bin'] = chi_bin
        outs[key], fig, ax = full_fit(sel_DA)
        params[key] = outs[key].params.valuesdict()

        # Save paths
        outPath.joinpath('full_linefits_v1').mkdir(exist_ok=True)
        savePath = outPath.joinpath('full_linefits_v1', f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
        savePath.mkdir(exist_ok=True)
        sampleFitPath = savePath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}')
        sampleFitPath.mkdir(exist_ok=True)
                                          
        # Save plot 
        fig.savefig(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_chi{key}_pi-pi_fit.png'), dpi=150)

        plt.show()
        plt.close('all')
                         
    # Save fit params:
    json_data = json.dumps(params)
    with open(str(sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_params.json')), 'w') as f:
        f.write(json_data)   
                                          
    # Save fit results
    for chi_bin, out in outs.items():
        with sampleFitPath.joinpath(f'{sn_id[DA.sample_ID]}_{DA.sample_pos}_{DA.incident_angle}_{chi_bin}_fit_result.txt').open(mode='w') as f:
            f.write(out.fit_report())
        
    # Append to full in-memory dictionaries
    all_outs[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}'] = outs
    all_params[f'{sn_id[DA.sample_ID]}_{DA.sample_pos}'] = params

# # Save model_params.json in save folder for all samples
# json_data = json.dumps(all_params)
# with open(str(savePath.joinpath('model_params.json')), 'w') as f:
#     f.write(json_data)

# # Create full fit reports folder and save entire report .txt files inside:
# savePath.joinpath('fit_reports').mkdir(exist_ok=True)
# for film, outs in all_outs.items():
#     for chi_bin, out in outs.items():
#         with savePath.joinpath('fit_reports', f'{film}_{chi_bin}_fit_result.txt').open(mode='w') as f:
#             f.write(out.fit_report())

### Load saved fit results

In [ ]:
fitsPath = pathlib.Path('/nsls2/data/cms/proposals/2023-3/pass-311415/AL_2024C2/processed_data/full_linefits_v1/chiWidth-4_chiBins-18_chiRange10-82/')
[f.name for f in fitsPath.glob('PM6*')]

In [ ]:
# # Combine json files, thanks GPT:
# import json
# import os

# json1 = '/nsls2/data/cms/proposals/2023-3/pass-311415/AL_2024C2/processed_data/full_linefits_v1/chiWidth-4_chiBins-18_chiRange10-82/model_params.json'
# json2 = '/nsls2/data/cms/proposals/2023-3/pass-311415/AL_2024C2/processed_data/full_linefits_v1/chiWidth-4_chiBins-18_chiRange10-82/model_params_5CN.json'

# # Load the two JSON files
# with open(json1, 'r') as f1:
#     json_data_1 = json.load(f1)

# with open(json2, 'r') as f2:
#     json_data_2 = json.load(f2)

# # Merge the dictionaries
# all_params = {**json_data_1, **json_data_2}

# # Save each sample as an individual JSON file
# for sample_name, sample_data in all_params.items():
#     samplePath = fitsPath.joinpath(sample_name)
#     savePath = samplePath.joinpath(f'{sample_name}_params.json')
#     with open(savePath, 'w') as sample_file:
#         json.dump(sample_data, sample_file, indent=4)

In [ ]:
# Load jsons into one dict, from each filepath
all_params = {}
for sampleFitPath in fitsPath.glob('PM6*th0.110*'):
    sample_name = sampleFitPath.name
    jsonPath = sampleFitPath.joinpath(f'{sample_name}_params.json')
    with open(jsonPath, 'r') as json_file:
        params = json.load(json_file)
    all_params[sample_name] = params
    
all_params.keys()

In [ ]:
all_params['PM6_1CN-CF_x0.000_th0.110']['10-14']

In [ ]:
# Plot pseudovoigt fractions, d-spacings, scherrer coherence lengths, and normalized peak areas vs chi
%matplotlib inline
plt.close('all')

# Initialize figures to be populated later
fig_psvoigt, ax_psvoigt = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_dspacing, ax_dspacing = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_ccl, ax_ccl = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
fig_pole, ax_pole = plt.subplots(figsize=((5.5,2.5)), dpi=150, tight_layout=True)
# fig_pole, ax_pole = plt.subplots(figsize=((2.8,3.3)), dpi=150, tight_layout=True)

# Choose sample sets (by folder/sample name):
peak = 'lamella'
incident_angle = 'th0.110'
PM6_CF_samples = [f'PM6_0CN-CF_x0.000_{incident_angle}', f'PM6_p5CN-CF_x0.000_{incident_angle}', f'PM6_1CN-CF_x2.000_{incident_angle}', f'PM6_5CN-CF_x2.000_{incident_angle}']
PM6_CB_samples = [f'PM6_0CN-CB_x0.000_{incident_angle}', f'PM6_p5CN-CB_x0.000_{incident_angle}', f'PM6_1CN-CB_x0.000_{incident_angle}', f'PM6_5CN-CB_x0.000_{incident_angle}']


# Define custom colors for each film, here I've chosen just two shades along the same sequention colorbar for each sample group
colors = plt.cm.viridis_r(np.linspace(0,1,len(PM6_CF_samples)))

### Extracting info from all_params & populating ___ vs chi plots:
summed_peak_areas = {}
avg_dspacings = {}
avg_peak_centers = {}
avg_coherence_lengths = {}
# for i, film in enumerate(all_params.keys()):
for i, film in enumerate(PM6_CF_samples):
# for i, film in enumerate(PM6_CB_samples):
    peak_areas = []
    pvoigt_fracs = []
    peak_centers = []
    peak_fwhms = []
    
    chis = []
    for chi_bin, params in all_params[film].items():   
        # Get starting value of chi bin
        chi = int(chi_bin.split('-')[0]) + 2  # set to midpoint of values
        chis.append(chi)
        # Get peak areas
        peak_area = params[f'{peak}_amplitude']
        peak_areas.append(peak_area)
        # Get pseudovoigt fraction
        pvoigt_frac = params[f'{peak}_fraction']
        pvoigt_fracs.append(pvoigt_frac)
        # Get peak centers
        peak_center = params[f'{peak}_center']
        peak_centers.append(peak_center)
        # Get peak fwhms
        peak_fwhm = params[f'{peak}_fwhm']
        peak_fwhms.append(peak_fwhm)

    # Get normalized peak areas (divide by sum of all peak areas; adds to 1) and write area sum to dict
    peak_areas = np.array(peak_areas)
    summed_peak_areas[film] = np.sum(peak_areas)  # Write sum of peak area out, this corresponds to relative extent of crystallinity if thickness normalized
    normed_peak_areas = peak_areas / np.sum(peak_areas)

    # Calculate d-spacings from peak centers, calculate peak-area-weighted average of d-spacing
    dspacings = (2*np.pi) / np.array(peak_centers) 
    avg_peak_centers[film] = peak_center
    avg_dspacing = np.round(np.sum(dspacings*normed_peak_areas), 2)
    avg_dspacings[film] = avg_dspacing  

    # Calculate coherence length from peak fwhm, calculate peak-area-weighted average of coherence length    
    coherence_lengths = (2*np.pi*0.9) / np.array(peak_fwhms) 
    avg_coherence_length = np.round(np.sum(coherence_lengths*normed_peak_areas), 2)
    avg_coherence_lengths[film] = avg_coherence_length

    ## Plotting
    # Plot pseudovoigt fraction vs chi for each film
    ax_psvoigt.errorbar(chis, pvoigt_fracs, label=film, color=colors[i], marker='o')
    ax_psvoigt.set(title=f'{peak} pseudo-voigt lorentzian-to-gaussian ratio versus $\\chi$',
                   ylabel='Lorentzian peak fraction',
                   xlabel='Binned $\\chi$ value [°]')
    ax_psvoigt.legend(title='Film', loc='upper left', bbox_to_anchor=(1,1))

    # Plot d-spacing vs chi for each film
    ax_dspacing.errorbar(chis, dspacings, label=film, color=colors[i], marker='o', 
                         capsize=4)
    ax_dspacing.set(title=f'{peak} d-spacing versus $\\chi$',
                    ylabel='d-spacing [Å]',
                    xlabel='$\\chi$ value [°]')
                    # ylim=(3.55,3.8))
    ax_dspacing.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))
    ax_dspacing.grid(visible=True,which='major',axis='y')

    # Plot coherence length vs chi for each film
    ax_ccl.errorbar(chis, coherence_lengths, label=film, color=colors[i], marker='o', 
                    capsize=4)
    ax_ccl.set(title=f'{peak} crystalline coherence length (CCL) versus $\\chi$',
               ylabel='CCL [Å]',
               xlabel='$\\chi$ value [°]')
               # ylim=(13,18.5))
    ax_ccl.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))
    ax_ccl.grid(visible=True,which='major',axis='y')
    
    # Plot 'pole figure' for each film
    ax_pole.errorbar(chis, normed_peak_areas, label=film, color=colors[i], marker='o',
                     capsize=4)
    ax_pole.set(title=f'{peak} pole figure',
                ylabel='Peak area [arb. units]',
                xlabel='$\\chi$ value [°]')
    # ax_pole.set_ybound(lower=0, upper=0.21)
    ax_pole.grid(visible=True,which='major',axis='y')
    # ax_pole.legend(title='Film')    
    ax_pole.legend(title='Film', loc='upper left', bbox_to_anchor=(1,0.85))

    # Extract chi values for file saving purposes
    chi_width = chis[0]-chis[1]
    chi_bins = len(chis)
    chi_min = chis[-1] - 1
    chi_max = chis[0]+chi_width - 1

    # # Save
    # parent_folder = 'full_chi_fit_results_plots_v1'
    # outPath.joinpath(parent_folder).mkdir(exist_ok=True)
    # savePath = outPath.joinpath(parent_folder, f'chiWidth-{chi_width}_chiBins-{chi_bins}_chiRange{chi_min}-{chi_max}')
    # savePath.mkdir(exist_ok=True)
    # fig_psvoigt.savefig(savePath.joinpath(f'pseudovoigt_ratios_pi-pi_fit.png'), dpi=150)
    # fig_dspacing.savefig(savePath.joinpath(f'dspacings_pi-pi_fit.png'), dpi=150)
    # fig_ccl.savefig(savePath.joinpath(f'coherence_lengths_pi-pi_fit.png'), dpi=150)
    # fig_pole.savefig(savePath.joinpath(f'peak_areas_pi-pi_fit.png'), dpi=150)
    # # fig_pole.savefig(savePath.joinpath(f'peak_areas_pi-pi_fit_narrow.png'), dpi=150)
    
plt.show()
plt.close('all')  